# SEC EDGAR - Filings' Data Extraction Part 2

## Import Libraries

In [1]:
# import necessary libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
from collections import Counter

# note: the requests library and the BeautifulSoup library will be used for web scraping
# note: the collections library will be used to identify the most common items in lists

## Read Dataset to Pandas Dataframe

In [2]:
# read filing_page.csv file into a pandas dataframe
filing_page_df = pd.read_csv('filing_page.csv')
# display filing_page_df
display(filing_page_df.head())

,CIK,Ticker_Symbol,Company_Name,Data_Date,Filing_Date,Filing_Type,Filing_Url,MDA_Placement,Filing_Check,Filing_Report_Url
0,1750,AIR,AAR CORP,2012-05-31,2012-07-19,10-K,https://www.sec.gov/Archives/edgar/data/1750/0...,10-K,True,https://www.sec.gov/Archives/edgar/data/1750/0...
1,1750,AIR,AAR CORP,2013-05-31,2013-07-26,10-K,https://www.sec.gov/Archives/edgar/data/1750/0...,10-K,True,https://www.sec.gov/Archives/edgar/data/1750/0...
2,1750,AIR,AAR CORP,2014-05-31,2014-07-17,10-K,https://www.sec.gov/Archives/edgar/data/1750/0...,10-K,True,https://www.sec.gov/Archives/edgar/data/1750/0...
3,1750,AIR,AAR CORP,2016-05-31,2016-07-13,10-K,https://www.sec.gov/Archives/edgar/data/1750/0...,10-K,True,https://www.sec.gov/Archives/edgar/data/1750/0...
4,1750,AIR,AAR CORP,2019-05-31,2019-07-18,10-K,https://www.sec.gov/Archives/edgar/data/1750/0...,10-K,True,https://www.sec.gov/Archives/edgar/data/1750/0...


## 10-K Filings - Source Management's Discussion and Analysis Narrative Page Numbers (Start and End)

In [3]:
# create a dataframe containing 10-K observations
# note: observations will be filtered from filing_page_df
filing_10k_mda_df = filing_page_df.loc[(filing_page_df['Filing_Type']=='10-K') | (filing_page_df['Filing_Type']=='10-KT')].reset_index(drop=True)

# find page number on which management's discussion and analysis narrative starts and ends
# note: the page numbers (start and end) will be stored in new columns of filing_10k_mda_df
# note: the management's discussion and analysis narrative is labeled as 'item 7' in 10-K filings
# note: the end of the management's discussion and analysis narrative is the start of the next item (i.e. item 7a)
filing_10k_mda_df['MDA_Page_Number_Start'] = ''
filing_10k_mda_df['MDA_Page_Number_End'] = ''
for idx, row in filing_10k_mda_df.iterrows():
    filing_request = requests.get(url=row['Filing_Report_Url'])
    filing_soup = BeautifulSoup(filing_request.content, 'lxml')
    mda_regex = re.compile(r'(?i)management(.|)s\s*discussion\s*(and|\&)\s*analysis.*')
    qqd_regex = re.compile(r'(?i)quantitative\s*(and|\&)\s*qualitative\s*disclosures.*')
    mda_occurence = filing_soup.find_all(text=mda_regex)
    qqd_occurence = filing_soup.find_all(text=qqd_regex)
    if len(mda_occurence)!=0:
        mda_table = None
        i = 0
        while mda_table==None:
            if i==len(mda_occurence):
                break
            else:
                mda_contents_page_row = mda_occurence[i].find_parent('tr')
                i += 1
                mda_table = mda_contents_page_row
        if mda_contents_page_row!=None:
            mda_contents_page_row_numbers = re.findall('\d+', mda_contents_page_row.text)
            if len(mda_contents_page_row_numbers)>1:
                for number in mda_contents_page_row_numbers:
                    if number=='7':
                        continue
                    else:
                        filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start'] = number
                        break
            elif len(mda_contents_page_row_numbers)==1 and mda_contents_page_row_numbers[0]!='7':
                filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start'] = mda_contents_page_row_numbers[0]
            else:
                filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start'] = 'N/A'                
        else:
            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start'] = 'N/A'
    else:
        filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start'] = 'N/A'

    if len(qqd_occurence)!=0:
        qqd_table = None
        i = 0
        while qqd_table==None:
            if i==len(qqd_occurence):
                break
            else:
                qqd_contents_page_row = qqd_occurence[i].find_parent('tr')
                i += 1
                qqd_table = qqd_contents_page_row
        if qqd_contents_page_row!=None:
            qqd_contents_page_row_numbers = re.findall('\d+', qqd_contents_page_row.text)
            if len(qqd_contents_page_row_numbers)>1:
                for number in qqd_contents_page_row_numbers:
                    if number=='7':
                        continue
                    else:
                        filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End'] = number
                        break
            elif len(qqd_contents_page_row_numbers)==1 and qqd_contents_page_row_numbers[0]!='7':
                filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End'] = qqd_contents_page_row_numbers[0]
            else:
                filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End'] = 'N/A'                        
        else:
            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End'] = 'N/A'
    else:
        filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End'] = 'N/A'
        
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(idx, row['Ticker_Symbol'])
    print('filing_date:', row['Filing_Date'])
    print('management discussion and analysis narrative starts on page:', filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start'])
    print('management discussion and analysis narrative ends on page:', filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 1 second 
    time.sleep(1)

-------------------------------------------------------------------------------------
0 AIR
filing_date: 2012-07-19
management discussion and analysis narrative starts on page: 17
management discussion and analysis narrative ends on page: 29


-------------------------------------------------------------------------------------
1 AIR
filing_date: 2013-07-26
management discussion and analysis narrative starts on page: 18
management discussion and analysis narrative ends on page: 27


-------------------------------------------------------------------------------------
2 AIR
filing_date: 2014-07-17
management discussion and analysis narrative starts on page: 19
management discussion and analysis narrative ends on page: 29


-------------------------------------------------------------------------------------
3 AIR
filing_date: 2016-07-13
management discussion and analysis narrative starts on page: 20
management discussion and analysis narrative ends on page: 33


------------------------

-------------------------------------------------------------------------------------
34 ADM
filing_date: 2018-02-16
management discussion and analysis narrative starts on page: 26
management discussion and analysis narrative ends on page: 46


-------------------------------------------------------------------------------------
35 ADM
filing_date: 2019-02-19
management discussion and analysis narrative starts on page: 27
management discussion and analysis narrative ends on page: 49


-------------------------------------------------------------------------------------
36 SWN
filing_date: 2016-02-25
management discussion and analysis narrative starts on page: 45
management discussion and analysis narrative ends on page: 61


-------------------------------------------------------------------------------------
37 SWN
filing_date: 2019-02-28
management discussion and analysis narrative starts on page: 49
management discussion and analysis narrative ends on page: 70


--------------------

-------------------------------------------------------------------------------------
68 CTL
filing_date: 2019-03-11
management discussion and analysis narrative starts on page: 53
management discussion and analysis narrative ends on page: 81


-------------------------------------------------------------------------------------
69 CSS
filing_date: 2011-05-26
management discussion and analysis narrative starts on page: 13
management discussion and analysis narrative ends on page: 22


-------------------------------------------------------------------------------------
70 CSS
filing_date: 2012-06-04
management discussion and analysis narrative starts on page: 13
management discussion and analysis narrative ends on page: 21


-------------------------------------------------------------------------------------
71 CSS
filing_date: 2013-05-31
management discussion and analysis narrative starts on page: 14
management discussion and analysis narrative ends on page: 23


--------------------

-------------------------------------------------------------------------------------
102 KODK
filing_date: 2019-04-01
management discussion and analysis narrative starts on page: 28
management discussion and analysis narrative ends on page: 43


-------------------------------------------------------------------------------------
103 EBF
filing_date: 2011-05-10
management discussion and analysis narrative starts on page: 16
management discussion and analysis narrative ends on page: 26


-------------------------------------------------------------------------------------
104 EBF
filing_date: 2012-05-11
management discussion and analysis narrative starts on page: 16
management discussion and analysis narrative ends on page: 26


-------------------------------------------------------------------------------------
105 EBF
filing_date: 2013-05-10
management discussion and analysis narrative starts on page: 17
management discussion and analysis narrative ends on page: 27


---------------

-------------------------------------------------------------------------------------
136 LXU
filing_date: 2016-02-29
management discussion and analysis narrative starts on page: 27
management discussion and analysis narrative ends on page: 52


-------------------------------------------------------------------------------------
137 LXU
filing_date: 2017-02-27
management discussion and analysis narrative starts on page: 26
management discussion and analysis narrative ends on page: 46


-------------------------------------------------------------------------------------
138 LXU
filing_date: 2018-02-26
management discussion and analysis narrative starts on page: 28
management discussion and analysis narrative ends on page: 46


-------------------------------------------------------------------------------------
139 LXU
filing_date: 2019-02-26
management discussion and analysis narrative starts on page: 28
management discussion and analysis narrative ends on page: 45


----------------

...

-------------------------------------------------------------------------------------
952 TLYS
filing_date: 2016-03-30
management discussion and analysis narrative starts on page: 32
management discussion and analysis narrative ends on page: 43


-------------------------------------------------------------------------------------
953 SLCA
filing_date: 2019-02-20
management discussion and analysis narrative starts on page: 60
management discussion and analysis narrative ends on page: 80


-------------------------------------------------------------------------------------
954 ERA
filing_date: 2016-02-26
management discussion and analysis narrative starts on page: 33
management discussion and analysis narrative ends on page: 48


-------------------------------------------------------------------------------------
955 ERA
filing_date: 2017-03-09
management discussion and analysis narrative starts on page: 36
management discussion and analysis narrative ends on page: 53


--------------

-------------------------------------------------------------------------------------
986 MPLX
filing_date: 2017-02-24
management discussion and analysis narrative starts on page: 77
management discussion and analysis narrative ends on page: 109


-------------------------------------------------------------------------------------
987 SUN
filing_date: 2016-02-26
management discussion and analysis narrative starts on page: 36
management discussion and analysis narrative ends on page: 51


-------------------------------------------------------------------------------------
988 SUN
filing_date: 2017-02-24
management discussion and analysis narrative starts on page: 34
management discussion and analysis narrative ends on page: 48


-------------------------------------------------------------------------------------
989 CWEN
filing_date: 2016-02-29
management discussion and analysis narrative starts on page: 41
management discussion and analysis narrative ends on page: 62


-------------

-------------------------------------------------------------------------------------
1020 PRTY
filing_date: 2019-02-28
management discussion and analysis narrative starts on page: 37
management discussion and analysis narrative ends on page: 63


-------------------------------------------------------------------------------------
1021 PE
filing_date: 2019-02-27
management discussion and analysis narrative starts on page: 62
management discussion and analysis narrative ends on page: 82


-------------------------------------------------------------------------------------
1022 RUBI
filing_date: 2017-03-15
management discussion and analysis narrative starts on page: 50
management discussion and analysis narrative ends on page: 74


-------------------------------------------------------------------------------------
1023 RUBI
filing_date: 2018-03-15
management discussion and analysis narrative starts on page: 55
management discussion and analysis narrative ends on page: 77


----------

-------------------------------------------------------------------------------------
1054 VSTO
filing_date: 2018-05-18
management discussion and analysis narrative starts on page: 31
management discussion and analysis narrative ends on page: 47


-------------------------------------------------------------------------------------
1055 VSTO
filing_date: 2019-05-23
management discussion and analysis narrative starts on page: 27
management discussion and analysis narrative ends on page: 39


-------------------------------------------------------------------------------------
1056 JAX
filing_date: 2019-03-14
management discussion and analysis narrative starts on page: 36
management discussion and analysis narrative ends on page: 49


-------------------------------------------------------------------------------------
1057 QSR
filing_date: 2015-03-02
management discussion and analysis narrative starts on page: 31
management discussion and analysis narrative ends on page: 58


----------

-------------------------------------------------------------------------------------
1088 BH
filing_date: 2018-02-26
management discussion and analysis narrative starts on page: 14
management discussion and analysis narrative ends on page: 23


-------------------------------------------------------------------------------------
1089 BH
filing_date: 2019-02-25
management discussion and analysis narrative starts on page: 14
management discussion and analysis narrative ends on page: 24


-------------------------------------------------------------------------------------
1090 WRK
filing_date: 2011-11-23
management discussion and analysis narrative starts on page: 18
management discussion and analysis narrative ends on page: 37


-------------------------------------------------------------------------------------
1091 WRK
filing_date: 2015-11-27
management discussion and analysis narrative starts on page: 27
management discussion and analysis narrative ends on page: 49


--------------

## 10-K Filings - Manually Input Management's Discussion and Analysis Narrative Page Numbers (Start and End) for N/A Observations

In [4]:
# manually input N/A Observations for MDA_Page_Number_Start and MDA_Page_Number_End in filing_10k_mda_df
filing_10k_mda_df.loc[12, 'MDA_Page_Number_Start'] = '17'
filing_10k_mda_df.loc[12, 'MDA_Page_Number_End'] = '25'

filing_10k_mda_df.loc[13, 'MDA_Page_Number_Start'] = '17'
filing_10k_mda_df.loc[13, 'MDA_Page_Number_End'] = '25'

filing_10k_mda_df.loc[14, 'MDA_Page_Number_Start'] = '17'
filing_10k_mda_df.loc[14, 'MDA_Page_Number_End'] = '24'

filing_10k_mda_df.loc[30, 'MDA_Page_Number_Start'] = '24'

filing_10k_mda_df.loc[48, 'MDA_Page_Number_End'] = '40'

filing_10k_mda_df.loc[49, 'MDA_Page_Number_End'] = '39'

filing_10k_mda_df.loc[50, 'MDA_Page_Number_End'] = '39'

filing_10k_mda_df.loc[77, 'MDA_Page_Number_End'] = '25'

filing_10k_mda_df.loc[84, 'MDA_Page_Number_Start'] = '24'
filing_10k_mda_df.loc[84, 'MDA_Page_Number_End'] = '52'

filing_10k_mda_df.loc[87, 'MDA_Page_Number_Start'] = '26'
filing_10k_mda_df.loc[87, 'MDA_Page_Number_End'] = '72'

filing_10k_mda_df.loc[97, 'MDA_Page_Number_Start'] = '38'
filing_10k_mda_df.loc[97, 'MDA_Page_Number_End'] = '63'

filing_10k_mda_df.loc[98, 'MDA_Page_Number_Start'] = '33'

filing_10k_mda_df.loc[123, 'MDA_Page_Number_Start'] = '31'
filing_10k_mda_df.loc[123, 'MDA_Page_Number_End'] = '51'

filing_10k_mda_df.loc[124, 'MDA_Page_Number_Start'] = '31'
filing_10k_mda_df.loc[124, 'MDA_Page_Number_End'] = '51'

filing_10k_mda_df.loc[125, 'MDA_Page_Number_Start'] = '31'
filing_10k_mda_df.loc[125, 'MDA_Page_Number_End'] = '51'

filing_10k_mda_df.loc[126, 'MDA_Page_Number_Start'] = '26'
filing_10k_mda_df.loc[126, 'MDA_Page_Number_End'] = '43'

filing_10k_mda_df.loc[133, 'MDA_Page_Number_Start'] = '35'
filing_10k_mda_df.loc[133, 'MDA_Page_Number_End'] = '66'

filing_10k_mda_df.loc[134, 'MDA_Page_Number_Start'] = '33'
filing_10k_mda_df.loc[134, 'MDA_Page_Number_End'] = '63'

filing_10k_mda_df.loc[135, 'MDA_Page_Number_Start'] = '20'
filing_10k_mda_df.loc[135, 'MDA_Page_Number_End'] = '45'

filing_10k_mda_df.loc[142, 'MDA_Page_Number_End'] = '43'

filing_10k_mda_df.loc[143, 'MDA_Page_Number_Start'] = '19'
filing_10k_mda_df.loc[143, 'MDA_Page_Number_End'] = '40'

filing_10k_mda_df.loc[144, 'MDA_Page_Number_Start'] = '20'
filing_10k_mda_df.loc[144, 'MDA_Page_Number_End'] = '42'

filing_10k_mda_df.loc[145, 'MDA_Page_Number_Start'] = '20'
filing_10k_mda_df.loc[145, 'MDA_Page_Number_End'] = '46'

filing_10k_mda_df.loc[155, 'MDA_Page_Number_Start'] = '31'
filing_10k_mda_df.loc[155, 'MDA_Page_Number_End'] = '53'

filing_10k_mda_df.loc[156, 'MDA_Page_Number_Start'] = '34'
filing_10k_mda_df.loc[156, 'MDA_Page_Number_End'] = '60'

filing_10k_mda_df.loc[158, 'MDA_Page_Number_Start'] = '30'
filing_10k_mda_df.loc[158, 'MDA_Page_Number_End'] = '50'

filing_10k_mda_df.loc[159, 'MDA_Page_Number_Start'] = '28'
filing_10k_mda_df.loc[159, 'MDA_Page_Number_End'] = '46'

filing_10k_mda_df.loc[160, 'MDA_Page_Number_Start'] = '27'
filing_10k_mda_df.loc[160, 'MDA_Page_Number_End'] = '43'

filing_10k_mda_df.loc[161, 'MDA_Page_Number_Start'] = '28'
filing_10k_mda_df.loc[161, 'MDA_Page_Number_End'] = '54'

filing_10k_mda_df.loc[219, 'MDA_Page_Number_End'] = '43'

filing_10k_mda_df.loc[220, 'MDA_Page_Number_End'] = '36'

filing_10k_mda_df.loc[265, 'MDA_Page_Number_End'] = '76'

filing_10k_mda_df.loc[266, 'MDA_Page_Number_End'] = '81'

filing_10k_mda_df.loc[267, 'MDA_Page_Number_End'] = '72'

filing_10k_mda_df.loc[330, 'MDA_Page_Number_Start'] = '23'
filing_10k_mda_df.loc[330, 'MDA_Page_Number_End'] = '47'

filing_10k_mda_df.loc[331, 'MDA_Page_Number_Start'] = '23'
filing_10k_mda_df.loc[331, 'MDA_Page_Number_End'] = '48'

filing_10k_mda_df.loc[332, 'MDA_Page_Number_Start'] = '23'
filing_10k_mda_df.loc[332, 'MDA_Page_Number_End'] = '50'

filing_10k_mda_df.loc[333, 'MDA_Page_Number_Start'] = '23'
filing_10k_mda_df.loc[333, 'MDA_Page_Number_End'] = '47'

filing_10k_mda_df.loc[334, 'MDA_Page_Number_Start'] = '21'
filing_10k_mda_df.loc[334, 'MDA_Page_Number_End'] = '35'

filing_10k_mda_df.loc[335, 'MDA_Page_Number_Start'] = '22'
filing_10k_mda_df.loc[335, 'MDA_Page_Number_End'] = '35'

filing_10k_mda_df.loc[336, 'MDA_Page_Number_Start'] = '25'
filing_10k_mda_df.loc[336, 'MDA_Page_Number_End'] = '44'

filing_10k_mda_df.loc[340, 'MDA_Page_Number_Start'] = '45'
filing_10k_mda_df.loc[340, 'MDA_Page_Number_End'] = '66'

filing_10k_mda_df.loc[345, 'MDA_Page_Number_Start'] = '35'
filing_10k_mda_df.loc[345, 'MDA_Page_Number_End'] = '67'

filing_10k_mda_df.loc[346, 'MDA_Page_Number_Start'] = '37'
filing_10k_mda_df.loc[346, 'MDA_Page_Number_End'] = '67'

filing_10k_mda_df.loc[367, 'MDA_Page_Number_End'] = '23'

filing_10k_mda_df.loc[368, 'MDA_Page_Number_End'] = '20'

filing_10k_mda_df.loc[371, 'MDA_Page_Number_Start'] = '68'
filing_10k_mda_df.loc[371, 'MDA_Page_Number_End'] = '128'

filing_10k_mda_df.loc[372, 'MDA_Page_Number_Start'] = '72'
filing_10k_mda_df.loc[372, 'MDA_Page_Number_End'] = '136'

filing_10k_mda_df.loc[373, 'MDA_Page_Number_Start'] = '61'
filing_10k_mda_df.loc[373, 'MDA_Page_Number_End'] = '104'

filing_10k_mda_df.loc[377, 'MDA_Page_Number_Start'] = '34'

filing_10k_mda_df.loc[403, 'MDA_Page_Number_Start'] = '21'
filing_10k_mda_df.loc[403, 'MDA_Page_Number_End'] = '41'

filing_10k_mda_df.loc[410, 'MDA_Page_Number_Start'] = '29'

filing_10k_mda_df.loc[411, 'MDA_Page_Number_Start'] = '29'

filing_10k_mda_df.loc[412, 'MDA_Page_Number_Start'] = '29'
filing_10k_mda_df.loc[412, 'MDA_Page_Number_End'] = '39'

filing_10k_mda_df.loc[413, 'MDA_Page_Number_Start'] = '29'

filing_10k_mda_df.loc[414, 'MDA_Page_Number_Start'] = '31'

filing_10k_mda_df.loc[415, 'MDA_Page_Number_Start'] = '31'

filing_10k_mda_df.loc[427, 'MDA_Page_Number_Start'] = '20'
filing_10k_mda_df.loc[427, 'MDA_Page_Number_End'] = '20'

filing_10k_mda_df.loc[432, 'MDA_Page_Number_Start'] = '19'
filing_10k_mda_df.loc[432, 'MDA_Page_Number_End'] = '38'

filing_10k_mda_df.loc[474, 'MDA_Page_Number_Start'] = '41'
filing_10k_mda_df.loc[474, 'MDA_Page_Number_End'] = '88'

filing_10k_mda_df.loc[477, 'MDA_Page_Number_End'] = '69'

filing_10k_mda_df.loc[478, 'MDA_Page_Number_End'] = '71'

filing_10k_mda_df.loc[499, 'MDA_Page_Number_Start'] = '28'

filing_10k_mda_df.loc[500, 'MDA_Page_Number_Start'] = '26'

filing_10k_mda_df.loc[502, 'MDA_Page_Number_Start'] = '21'
filing_10k_mda_df.loc[502, 'MDA_Page_Number_End'] = '35'

filing_10k_mda_df.loc[503, 'MDA_Page_Number_Start'] = '18'
filing_10k_mda_df.loc[503, 'MDA_Page_Number_End'] = '33'

filing_10k_mda_df.loc[504, 'MDA_Page_Number_Start'] = '20'
filing_10k_mda_df.loc[504, 'MDA_Page_Number_End'] = '35'

filing_10k_mda_df.loc[505, 'MDA_Page_Number_Start'] = '19'
filing_10k_mda_df.loc[505, 'MDA_Page_Number_End'] = '38'

filing_10k_mda_df.loc[506, 'MDA_Page_Number_Start'] = '19'
filing_10k_mda_df.loc[506, 'MDA_Page_Number_End'] = '38'

filing_10k_mda_df.loc[507, 'MDA_Page_Number_Start'] = '19'
filing_10k_mda_df.loc[507, 'MDA_Page_Number_End'] = '36'

filing_10k_mda_df.loc[508, 'MDA_Page_Number_Start'] = '20'
filing_10k_mda_df.loc[508, 'MDA_Page_Number_End'] = '38'

filing_10k_mda_df.loc[509, 'MDA_Page_Number_Start'] = '22'
filing_10k_mda_df.loc[509, 'MDA_Page_Number_End'] = '41'

filing_10k_mda_df.loc[535, 'MDA_Page_Number_Start'] = '50'
filing_10k_mda_df.loc[535, 'MDA_Page_Number_End'] = '85'

filing_10k_mda_df.loc[536, 'MDA_Page_Number_Start'] = '55'
filing_10k_mda_df.loc[536, 'MDA_Page_Number_End'] = '93'

filing_10k_mda_df.loc[552, 'MDA_Page_Number_Start'] = '14'
filing_10k_mda_df.loc[552, 'MDA_Page_Number_End'] = '21'

filing_10k_mda_df.loc[553, 'MDA_Page_Number_Start'] = '11'
filing_10k_mda_df.loc[553, 'MDA_Page_Number_End'] = '18'

filing_10k_mda_df.loc[554, 'MDA_Page_Number_Start'] = '11'
filing_10k_mda_df.loc[554, 'MDA_Page_Number_End'] = '18'

filing_10k_mda_df.loc[564, 'MDA_Page_Number_Start'] = '37'
filing_10k_mda_df.loc[564, 'MDA_Page_Number_End'] = '58'

filing_10k_mda_df.loc[565, 'MDA_Page_Number_Start'] = '31'
filing_10k_mda_df.loc[565, 'MDA_Page_Number_End'] = '51'

filing_10k_mda_df.loc[566, 'MDA_Page_Number_Start'] = '26'
filing_10k_mda_df.loc[566, 'MDA_Page_Number_End'] = '44'

filing_10k_mda_df.loc[567, 'MDA_Page_Number_Start'] = '29'
filing_10k_mda_df.loc[567, 'MDA_Page_Number_End'] = '46'

filing_10k_mda_df.loc[649, 'MDA_Page_Number_End'] = '52'

filing_10k_mda_df.loc[650, 'MDA_Page_Number_End'] = '52'

filing_10k_mda_df.loc[651, 'MDA_Page_Number_End'] = '57'

filing_10k_mda_df.loc[689, 'MDA_Page_Number_End'] = '38'

filing_10k_mda_df.loc[770, 'MDA_Page_Number_Start'] = '40'
filing_10k_mda_df.loc[770, 'MDA_Page_Number_End'] = '62'

filing_10k_mda_df.loc[771, 'MDA_Page_Number_Start'] = '39'
filing_10k_mda_df.loc[771, 'MDA_Page_Number_End'] = '62'

filing_10k_mda_df.loc[772, 'MDA_Page_Number_Start'] = '52'
filing_10k_mda_df.loc[772, 'MDA_Page_Number_End'] = '77'

filing_10k_mda_df.loc[773, 'MDA_Page_Number_Start'] = '42'
filing_10k_mda_df.loc[773, 'MDA_Page_Number_End'] = '69'

filing_10k_mda_df.loc[774, 'MDA_Page_Number_Start'] = '43'
filing_10k_mda_df.loc[774, 'MDA_Page_Number_End'] = '75'

filing_10k_mda_df.loc[775, 'MDA_Page_Number_Start'] = '50'
filing_10k_mda_df.loc[775, 'MDA_Page_Number_End'] = '88'

filing_10k_mda_df.loc[776, 'MDA_Page_Number_Start'] = '53'
filing_10k_mda_df.loc[776, 'MDA_Page_Number_End'] = '89'

filing_10k_mda_df.loc[777, 'MDA_Page_Number_Start'] = '52'
filing_10k_mda_df.loc[777, 'MDA_Page_Number_End'] = '87'

filing_10k_mda_df.loc[797, 'MDA_Page_Number_End'] = '38'

filing_10k_mda_df.loc[808, 'MDA_Page_Number_End'] = '71'

filing_10k_mda_df.loc[809, 'MDA_Page_Number_End'] = '70'

filing_10k_mda_df.loc[810, 'MDA_Page_Number_End'] = '67'

filing_10k_mda_df.loc[811, 'MDA_Page_Number_End'] = '57'

filing_10k_mda_df.loc[812, 'MDA_Page_Number_End'] = '48'

filing_10k_mda_df.loc[813, 'MDA_Page_Number_End'] = '47'

filing_10k_mda_df.loc[814, 'MDA_Page_Number_End'] = '45'

filing_10k_mda_df.loc[914, 'MDA_Page_Number_End'] = '68'

filing_10k_mda_df.loc[929, 'MDA_Page_Number_End'] = '60'

filing_10k_mda_df.loc[930, 'MDA_Page_Number_End'] = '59'

filing_10k_mda_df.loc[931, 'MDA_Page_Number_End'] = '58'

filing_10k_mda_df.loc[933, 'MDA_Page_Number_Start'] = '58'

filing_10k_mda_df.loc[940, 'MDA_Page_Number_End'] = '72'

filing_10k_mda_df.loc[941, 'MDA_Page_Number_End'] = '72'

filing_10k_mda_df.loc[958, 'MDA_Page_Number_End'] = '76'

filing_10k_mda_df.loc[981, 'MDA_Page_Number_Start'] = '40'

filing_10k_mda_df.loc[1039, 'MDA_Page_Number_Start'] = '43'

filing_10k_mda_df.loc[1040, 'MDA_Page_Number_Start'] = '42'

filing_10k_mda_df.loc[1041, 'MDA_Page_Number_Start'] = '40'

filing_10k_mda_df.loc[1070, 'MDA_Page_Number_Start'] = '18'

filing_10k_mda_df.loc[1076, 'MDA_Page_Number_Start'] = '48'
filing_10k_mda_df.loc[1076, 'MDA_Page_Number_End'] = '78'

filing_10k_mda_df.loc[1077, 'MDA_Page_Number_Start'] = '37'
filing_10k_mda_df.loc[1077, 'MDA_Page_Number_End'] = '62'

filing_10k_mda_df.loc[1078, 'MDA_Page_Number_Start'] = '38'
filing_10k_mda_df.loc[1078, 'MDA_Page_Number_End'] = '63'

filing_10k_mda_df.loc[1079, 'MDA_Page_Number_Start'] = '37'
filing_10k_mda_df.loc[1079, 'MDA_Page_Number_End'] = '61'

filing_10k_mda_df.loc[1080, 'MDA_Page_Number_Start'] = '36'
filing_10k_mda_df.loc[1080, 'MDA_Page_Number_End'] = '57'

filing_10k_mda_df.loc[1081, 'MDA_Page_Number_Start'] = '34'
filing_10k_mda_df.loc[1081, 'MDA_Page_Number_End'] = '55'

# observation situated in index 314 has MDA_Page_Number_Start > MDA_Page_Number_End; therefore it is necessary to manually amend 
filing_10k_mda_df.loc[314, 'MDA_Page_Number_Start'] = '54'
filing_10k_mda_df.loc[314, 'MDA_Page_Number_End'] = '85'

# observation situated in index 1093 of filing_10k_mda_df has no page numbers, and therefore its MDA_Page_Number_Start and MDA_Page_Number_End will remain as 'N/A'

## 10-K Filings - Store Previous Page Number for MDA_Page_Number_Start and MDA_Page_Number_End

In [5]:
# note: in order to identify a boundary of where the management's discussion and analysis narrative begins and ends, 
#       it is necessary to store MDA_Page_Number_Start - 1 and MDA_Page_Number_End - 1 in filing_10k_mda_df

# create a new column in filing_10k_mda_df to store previous page number to MDA_Page_Number_Start
filing_10k_mda_df['MDA_Page_Number_Start_Prev'] = ''
# create a new column in filing_10k_mda_df to store previous page number to MDA_Page_Number_End
filing_10k_mda_df['MDA_Page_Number_End_Prev'] = ''

# compute MDA_Page_Number_Start_Prev from MDA_Page_Number_Start
for idx, row in filing_10k_mda_df.iterrows():
    if row['MDA_Page_Number_Start']!='N/A':
        filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start_Prev'] = str((int(float(row['MDA_Page_Number_Start'].strip())) - 1))
    else:
        filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start_Prev'] = 'N/A'
# compute MDA_Page_Number_End_Prev from MDA_Page_Number_End
for idx, row in filing_10k_mda_df.iterrows():
    if row['MDA_Page_Number_End']!='N/A':
        filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End_Prev'] = str((int(float(row['MDA_Page_Number_End'].strip())) - 1))
    else:
        filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End_Prev'] = 'N/A'

## 10-K Filings - Check if Page Breaks Exist and Store MD&A Start and End Tags

In [6]:
# note: in order to extract the management's discussion and analysis narrative of each 10-K filing by page number,
#       a check on if page breaks (separating pages) exist is necessary
# note: page breaks that separate pages in 10-K filings are usually denoted by a thematic break (<hr>)
# note: if a 10-K filing does not contain thematic breaks, the next best alternative is to find the tag style 'page-break-before:always'
# note: the alternative solution can be done via a regular expression

# create a new column in filing_10k_mda_df to store a boolean value corresponding if page breaks exist or not
filing_10k_mda_df['Page_Breaks'] = ''
# create a new column in filing_10k_mda_df to store the tag in which the MD&A narrative begins
filing_10k_mda_df['MDA_Page_Number_Start_Tag'] = ''
# create a new column in filing_10k_mda_df to store the tag of the previous page to where the MD&A narrative begins
filing_10k_mda_df['MDA_Page_Number_Start_Prev_Tag'] = ''
# create a new column in filing_10k_mda_df to store the tag in which the MD&A narrative ends
filing_10k_mda_df['MDA_Page_Number_End_Tag'] = ''
# create a new column in filing_10k_mda_df to store the tag of the previous page to where the MD&A narrative ends
filing_10k_mda_df['MDA_Page_Number_End_Prev_Tag'] = ''
# create a new column in filing_10k_mda_df to store a boolean value corresponding to if MD&A narrative start page matches retrieved value
filing_10k_mda_df['MDA_Page_Number_Start_Match'] = ''
# create a new column in filing_10k_mda_df to store a boolean value corresponding to if previous page to MD&A start page matches retrieved value
filing_10k_mda_df['MDA_Page_Number_Start_Prev_Match'] = ''
# create a new column in filing_10k_mda_df to store a boolean value corresponding to if MD&A narrative end page matches retrieved value
filing_10k_mda_df['MDA_Page_Number_End_Match'] = ''
# create a new column in filing_10k_mda_df to store a boolean value corresponding to if previous page to MD&A end page matches retrieved value
filing_10k_mda_df['MDA_Page_Number_End_Prev_Match'] = ''

# start page
for idx, row in filing_10k_mda_df.iterrows():
    filing_request = requests.get(url=row['Filing_Report_Url'])
    filing_soup = BeautifulSoup(filing_request.content, 'lxml')
    if row['MDA_Page_Number_Start']!='N/A' and row['MDA_Page_Number_End']!='N/A':
        page_break_tags_one = filing_soup.find_all('hr')     
        page_break_tags_two = filing_soup.find_all(style=re.compile(r'(?i).*page-break-(after|before)\s*:\s*always.*'))
        if len(page_break_tags_one)!=0:
            filing_10k_mda_df.loc[idx, 'Page_Breaks'] = 'True'
            page_break_tag_counter = Counter(page_break_tags_one)
            most_common_tag = page_break_tag_counter.most_common(1)[0][0]
            for tag in page_break_tags_one:
                if tag==most_common_tag:
                    page_number = tag.find_previous(text=re.compile('^([^a-zA-Z0-9]|){1}\d{1,3}([^a-zA-Z0-9]|){1}$'))
                    if page_number!=None:
                        if page_number.strip()==row['MDA_Page_Number_Start'].strip():
                            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start_Tag'] = str(page_number.parent)
                            break
                        else:
                            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start_Tag'] = 'N/A'
                    else:
                        continue
                else:
                    pass
        elif len(page_break_tags_two)!=0:
            filing_10k_mda_df.loc[idx, 'Page_Breaks'] = 'True'
            page_break_tag_counter = Counter(page_break_tags_two)
            most_common_tag = page_break_tag_counter.most_common(1)[0][0]
            for tag in page_break_tags_two:
                if tag==most_common_tag:
                    page_number = tag.find_previous(text=re.compile('^([^a-zA-Z0-9]|){1}\d{1,3}([^a-zA-Z0-9]|){1}$'))
                    if page_number!=None:
                        if page_number.strip()==row['MDA_Page_Number_Start'].strip():
                            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start_Tag'] = str(page_number.parent)
                            break
                        else:
                            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start_Tag'] = 'N/A'
                    else:
                        continue
                else:
                    pass
        else:
            filing_10k_mda_df.loc[idx, 'Page_Breaks'] = 'False'
            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start_Tag'] = 'N/A' 
    else:
        filing_10k_mda_df.loc[idx, 'Page_Breaks'] = 'False'
        filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start_Tag'] = 'N/A'   
    
    if page_number.strip()==row['MDA_Page_Number_Start'].strip():
        filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start_Match'] = 'True'
    else:
        filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start_Match'] = 'False'

    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(idx, row['Ticker_Symbol'])
    print('\n')
    print('start page:')
    print('filing_date:', row['Filing_Date'])
    print('page breaks exist:', filing_10k_mda_df.loc[idx, 'Page_Breaks'])
    print('management\'s discussion and analysis start page number tag:', page_number.parent.name)
    print('observed management\'s discussion and analysis start page number:', row['MDA_Page_Number_Start'].strip())
    print('retrieved management\'s discussion and analysis start page number:', page_number.strip())
    print('md&a page number match:', filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start_Match'])
    print('\n')
    
    
    # previous page of start page
    if row['MDA_Page_Number_Start']!='N/A' and row['MDA_Page_Number_End']!='N/A':
        page_break_tags_one = filing_soup.find_all('hr')     
        page_break_tags_two = filing_soup.find_all(style=re.compile(r'(?i).*page-break-(after|before)\s*:\s*always.*'))
        if len(page_break_tags_one)!=0:
            page_break_tag_counter = Counter(page_break_tags_one)
            most_common_tag = page_break_tag_counter.most_common(1)[0][0]
            for tag in page_break_tags_one:
                if tag==most_common_tag:
                    page_number = tag.find_previous(text=re.compile('^([^a-zA-Z0-9]|){1}\d{1,3}([^a-zA-Z0-9]|){1}$'))
                    if page_number!=None:
                        if page_number.strip()==row['MDA_Page_Number_Start_Prev'].strip():
                            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start_Prev_Tag'] = str(page_number.parent)
                            break
                        else:
                            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start_Prev_Tag'] = 'N/A'
                    else:
                        continue
                else:
                    pass
        elif len(page_break_tags_two)!=0:
            page_break_tag_counter = Counter(page_break_tags_two)
            most_common_tag = page_break_tag_counter.most_common(1)[0][0]
            for tag in page_break_tags_two:
                if tag==most_common_tag:
                    page_number = tag.find_previous(text=re.compile('^([^a-zA-Z0-9]|){1}\d{1,3}([^a-zA-Z0-9]|){1}$'))
                    if page_number!=None:
                        if page_number.strip()==row['MDA_Page_Number_Start_Prev'].strip():
                            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start_Prev_Tag'] = str(page_number.parent)
                            break
                        else:
                            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start_Prev_Tag'] = 'N/A'
                    else:
                        continue
                else:
                    pass
        else:
            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start_Prev_Tag'] = 'N/A' 
    else:
        filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start_Prev_Tag'] = 'N/A'   
    
    if page_number.strip()==row['MDA_Page_Number_Start_Prev'].strip():
        filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start_Prev_Match'] = 'True'
    else:
        filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start_Prev_Match'] = 'False'

    # provide user feedback
    print('previous page to start page:')
    print('management\'s discussion and analysis previous page to start page number tag:', page_number.parent.name)
    print('observed management\'s discussion and analysis previous page to  start page number:', row['MDA_Page_Number_Start_Prev'].strip())
    print('retrieved management\'s discussion and analysis previous page to  start page number:', page_number.strip())
    print('md&a previous page to start page number match:', filing_10k_mda_df.loc[idx, 'MDA_Page_Number_Start_Prev_Match'])
    print('\n')
    
    
    # end page
    if row['MDA_Page_Number_Start']!='N/A' and row['MDA_Page_Number_End']!='N/A':
        page_break_tags_one = filing_soup.find_all('hr')     
        page_break_tags_two = filing_soup.find_all(style=re.compile(r'(?i).*page-break-(after|before)\s*:\s*always.*'))
        if len(page_break_tags_one)!=0:
            page_break_tag_counter = Counter(page_break_tags_one)
            most_common_tag = page_break_tag_counter.most_common(1)[0][0]
            for tag in page_break_tags_one:
                if tag==most_common_tag:
                    page_number = tag.find_previous(text=re.compile('^([^a-zA-Z0-9]|){1}\d{1,3}([^a-zA-Z0-9]|){1}$'))
                    if page_number!=None:
                        if page_number.strip()==row['MDA_Page_Number_End'].strip():
                            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End_Tag'] = str(page_number.parent)
                            break
                        else:
                            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End_Tag'] = 'N/A'
                    else:
                        continue
                else:
                    pass
        elif len(page_break_tags_two)!=0:
            page_break_tag_counter = Counter(page_break_tags_two)
            most_common_tag = page_break_tag_counter.most_common(1)[0][0]
            for tag in page_break_tags_two:
                if tag==most_common_tag:
                    page_number = tag.find_previous(text=re.compile('^([^a-zA-Z0-9]|){1}\d{1,3}([^a-zA-Z0-9]|){1}$'))
                    if page_number!=None:
                        if page_number.strip()==row['MDA_Page_Number_End'].strip():
                            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End_Tag'] = str(page_number.parent)
                            break
                        else:
                            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End_Tag'] = 'N/A'
                    else:
                        continue
                else:
                    pass
        else:
            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End_Tag'] = 'N/A' 
    else:
        filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End_Tag'] = 'N/A'
    
    if page_number.strip()==row['MDA_Page_Number_End'].strip():
        filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End_Match'] = 'True'
    else:
        filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End_Match'] = 'False'
    
    # provide user feedback
    print('end page:')
    print('management\'s discussion and analysis end page number tag:', page_number.parent.name)
    print('observed management\'s discussion and analysis end page number:', row['MDA_Page_Number_End'].strip())
    print('retrieved management\'s discussion and analysis end page number:', page_number.strip())
    print('md&a end page number match:', filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End_Match'])
    print('\n')
    
    
    # previous page to end page
    if row['MDA_Page_Number_Start']!='N/A' and row['MDA_Page_Number_End']!='N/A':
        page_break_tags_one = filing_soup.find_all('hr')     
        page_break_tags_two = filing_soup.find_all(style=re.compile(r'(?i).*page-break-(after|before)\s*:\s*always.*'))
        if len(page_break_tags_one)!=0:
            page_break_tag_counter = Counter(page_break_tags_one)
            most_common_tag = page_break_tag_counter.most_common(1)[0][0]
            for tag in page_break_tags_one:
                if tag==most_common_tag:
                    page_number = tag.find_previous(text=re.compile('^([^a-zA-Z0-9]|){1}\d{1,3}([^a-zA-Z0-9]|){1}$'))
                    if page_number!=None:
                        if page_number.strip()==row['MDA_Page_Number_End_Prev'].strip():
                            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End_Prev_Tag'] = str(page_number.parent)
                            break
                        else:
                            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End_Prev_Tag'] = 'N/A'
                    else:
                        continue
                else:
                    pass
        elif len(page_break_tags_two)!=0:
            page_break_tag_counter = Counter(page_break_tags_two)
            most_common_tag = page_break_tag_counter.most_common(1)[0][0]
            for tag in page_break_tags_two:
                if tag==most_common_tag:
                    page_number = tag.find_previous(text=re.compile('^([^a-zA-Z0-9]|){1}\d{1,3}([^a-zA-Z0-9]|){1}$'))
                    if page_number!=None:
                        if page_number.strip()==row['MDA_Page_Number_End_Prev'].strip():
                            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End_Prev_Tag'] = str(page_number.parent)
                            break
                        else:
                            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End_Prev_Tag'] = 'N/A'
                    else:
                        continue
                else:
                    pass
        else:
            filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End_Prev_Tag'] = 'N/A' 
    else:
        filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End_Prev_Tag'] = 'N/A'
    
    if page_number.strip()==row['MDA_Page_Number_End_Prev'].strip():
        filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End_Prev_Match'] = 'True'
    else:
        filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End_Prev_Match'] = 'False'
    
    # provide user feedback
    print('previous page to end page:')
    print('management\'s discussion and analysis previous to end page number tag:', page_number.parent.name)
    print('observed management\'s discussion and analysis previous to end page number:', row['MDA_Page_Number_End_Prev'].strip())
    print('retrieved management\'s discussion and analysis previous to end page number:', page_number.strip())
    print('md&a previous to end page number match:', filing_10k_mda_df.loc[idx, 'MDA_Page_Number_End_Prev_Match'])
    print('\n')
    
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 1 second
    time.sleep(1)   

-------------------------------------------------------------------------------------
0 AIR


start page:
filing_date: 2012-07-19
page breaks exist: True
management's discussion and analysis start page number tag: font
observed management's discussion and analysis start page number: 17
retrieved management's discussion and analysis start page number: 17
md&a page number match: True


previous page to start page:
management's discussion and analysis previous page to start page number tag: font
observed management's discussion and analysis previous page to  start page number: 16
retrieved management's discussion and analysis previous page to  start page number: 16
md&a previous page to start page number match: True


end page:
management's discussion and analysis end page number tag: font
observed management's discussion and analysis end page number: 29
retrieved management's discussion and analysis end page number: 29
md&a end page number match: True


previous page to end page:
managem

end page:
management's discussion and analysis end page number tag: font
observed management's discussion and analysis end page number: 37
retrieved management's discussion and analysis end page number: 37
md&a end page number match: True


previous page to end page:
management's discussion and analysis previous to end page number tag: font
observed management's discussion and analysis previous to end page number: 36
retrieved management's discussion and analysis previous to end page number: 36
md&a previous to end page number match: True


-------------------------------------------------------------------------------------
7 HES


start page:
filing_date: 2014-02-28
page breaks exist: True
management's discussion and analysis start page number tag: font
observed management's discussion and analysis start page number: 21
retrieved management's discussion and analysis start page number: 21
md&a page number match: True


previous page to start page:
management's discussion and analysis 

previous page to start page:
management's discussion and analysis previous page to start page number tag: font
observed management's discussion and analysis previous page to  start page number: 16
retrieved management's discussion and analysis previous page to  start page number: 16
md&a previous page to start page number match: True


end page:
management's discussion and analysis end page number tag: font
observed management's discussion and analysis end page number: 25
retrieved management's discussion and analysis end page number: 25
md&a end page number match: True


previous page to end page:
management's discussion and analysis previous to end page number tag: font
observed management's discussion and analysis previous to end page number: 24
retrieved management's discussion and analysis previous to end page number: 24
md&a previous to end page number match: True


-------------------------------------------------------------------------------------
14 AP


start page:
filing_da

-------------------------------------------------------------------------------------
20 APA


start page:
filing_date: 2013-03-01
page breaks exist: True
management's discussion and analysis start page number tag: font
observed management's discussion and analysis start page number: 36
retrieved management's discussion and analysis start page number: 36
md&a page number match: True


previous page to start page:
management's discussion and analysis previous page to start page number tag: font
observed management's discussion and analysis previous page to  start page number: 35
retrieved management's discussion and analysis previous page to  start page number: 35
md&a previous page to start page number match: True


end page:
management's discussion and analysis end page number tag: font
observed management's discussion and analysis end page number: 66
retrieved management's discussion and analysis end page number: 66
md&a end page number match: True


previous page to end page:
manage

end page:
management's discussion and analysis end page number tag: font
observed management's discussion and analysis end page number: 37
retrieved management's discussion and analysis end page number: 37
md&a end page number match: True


previous page to end page:
management's discussion and analysis previous to end page number tag: font
observed management's discussion and analysis previous to end page number: 36
retrieved management's discussion and analysis previous to end page number: 36
md&a previous to end page number match: True


-------------------------------------------------------------------------------------
27 SSI


start page:
filing_date: 2017-04-04
page breaks exist: True
management's discussion and analysis start page number tag: font
observed management's discussion and analysis start page number: 26
retrieved management's discussion and analysis start page number: 26
md&a page number match: True


previous page to start page:
management's discussion and analysis

...

previous page to end page:
management's discussion and analysis previous to end page number tag: p
observed management's discussion and analysis previous to end page number: 72
retrieved management's discussion and analysis previous to end page number: 72
md&a previous to end page number match: True


-------------------------------------------------------------------------------------
1072 CNDT


start page:
filing_date: 2019-02-28
page breaks exist: True
management's discussion and analysis start page number tag: font
observed management's discussion and analysis start page number: 29
retrieved management's discussion and analysis start page number: 29
md&a page number match: True


previous page to start page:
management's discussion and analysis previous page to start page number tag: font
observed management's discussion and analysis previous page to  start page number: 28
retrieved management's discussion and analysis previous page to  start page number: 28
md&a previous page to 

previous page to start page:
management's discussion and analysis previous page to start page number tag: font
observed management's discussion and analysis previous page to  start page number: 37
retrieved management's discussion and analysis previous page to  start page number: 37
md&a previous page to start page number match: True


end page:
management's discussion and analysis end page number tag: font
observed management's discussion and analysis end page number: 63
retrieved management's discussion and analysis end page number: 63
md&a end page number match: True


previous page to end page:
management's discussion and analysis previous to end page number tag: font
observed management's discussion and analysis previous to end page number: 62
retrieved management's discussion and analysis previous to end page number: 62
md&a previous to end page number match: True


-------------------------------------------------------------------------------------
1079 HPR


start page:
filing

previous page to end page:
management's discussion and analysis previous to end page number tag: font
observed management's discussion and analysis previous to end page number: 25
retrieved management's discussion and analysis previous to end page number: 25
md&a previous to end page number match: True


-------------------------------------------------------------------------------------
1085 BH


start page:
filing_date: 2014-11-24
page breaks exist: True
management's discussion and analysis start page number tag: font
observed management's discussion and analysis start page number: 16
retrieved management's discussion and analysis start page number: -73-
md&a page number match: False


previous page to start page:
management's discussion and analysis previous page to start page number tag: font
observed management's discussion and analysis previous page to  start page number: 15
retrieved management's discussion and analysis previous page to  start page number: -73-
md&a previous pa

previous page to start page:
management's discussion and analysis previous page to start page number tag: font
observed management's discussion and analysis previous page to  start page number: 26
retrieved management's discussion and analysis previous page to  start page number: 26
md&a previous page to start page number match: True


end page:
management's discussion and analysis end page number tag: font
observed management's discussion and analysis end page number: 49
retrieved management's discussion and analysis end page number: 49
md&a end page number match: True


previous page to end page:
management's discussion and analysis previous to end page number tag: font
observed management's discussion and analysis previous to end page number: 48
retrieved management's discussion and analysis previous to end page number: 48
md&a previous to end page number match: True


-------------------------------------------------------------------------------------
1092 WRK


start page:
filing

previous page to end page:
management's discussion and analysis previous to end page number tag: font
observed management's discussion and analysis previous to end page number: 55
retrieved management's discussion and analysis previous to end page number: 55
md&a previous to end page number match: True




## 10-K Filings - Retrieve Position of MD&A Start and End Tags in Fillings

In [7]:
# note: the management's discussion and analysis narrative of each 10-K filing can be extracted by using the boundary of the start page,
#       previous to start page, end page, and previous to end page
# note: a regular expression can be used to find occurences of item 7 and item 7a;
#       the position of the occurence within the given boundary will be stored
# note: if there is more than one occurence within the given boundary, these will be ignored due to time constraints

# create a new column in filing_10k_mda_df to store the position of the start of the md&a narrative
filing_10k_mda_df['MDA_Start_Position'] = ''
# create a new column in filing_10k_mda_df to store the position of the end of the md&a narrative
filing_10k_mda_df['MDA_End_Position'] = ''

# loop over observations in filing_10k_mda_df
for idx, row in filing_10k_mda_df.iterrows():
    if (row['MDA_Page_Number_Start_Match']!='False' and row['MDA_Page_Number_Start_Prev_Match']!='False' and
        row['MDA_Page_Number_End_Match']!='False' and row['MDA_Page_Number_End_Prev_Match']!='False'):
        filing_request = requests.get(url=row['Filing_Report_Url'])
        filing_soup = BeautifulSoup(filing_request.content, 'lxml')
        filing_soup_string = str(filing_soup)
        # start position
        # occurences of MDA_Page_Number_Start_Tag in filing_soup_string
        mda_start_tag = [i.start() for i in re.finditer(row['MDA_Page_Number_Start_Tag'], filing_soup_string)]
        # occurences of MDA_Page_Number_Start_Prev_Tag in filing_soup_string
        mda_start_prev_tag = [i.start() for i in re.finditer(row['MDA_Page_Number_Start_Prev_Tag'], filing_soup_string)]
        if len(mda_start_tag)==1 and len(mda_start_prev_tag)==1:
            item_7_regex = re.compile(r'(?i)item\s*7(\.|\s*)')
            mda_parent_tag = [str(i.parent) for i in filing_soup.find_all(text=item_7_regex)]
            if len(mda_parent_tag)!=0:
                mda_parent_tag_position = []
                values_in_boundary = []
                for i in mda_parent_tag:
                    mda_parent_tag_position.append(filing_soup_string.find(i))
                for i in mda_parent_tag_position:
                    if i > mda_start_prev_tag[0] and i < mda_start_tag[0]:
                        values_in_boundary.append(i)
                    else:
                        continue
                if len(values_in_boundary)==1:
                    filing_10k_mda_df.loc[idx, 'MDA_Start_Position'] = values_in_boundary[0]
                else:
                    filing_10k_mda_df.loc[idx, 'MDA_Start_Position'] = 'N/A'
            else:
                filing_10k_mda_df.loc[idx, 'MDA_Start_Position'] = 'N/A'
        elif len(mda_start_tag)==1 and len(mda_start_prev_tag)>1:
            difference = []
            for count, i in enumerate(mda_start_prev_tag):
                difference.append(mda_start_tag[0] - mda_start_prev_tag[count])
            difference_pos = []
            for count, i in enumerate(difference):
                if i > 0:
                    difference_pos.append(i)
                else:
                    continue
            if len(difference_pos)>0:
                mda_start_prev_tag = [mda_start_prev_tag[difference.index(min(difference_pos))]]                
                item_7_regex = re.compile(r'(?i)item\s*7(\.|\s*)')
                mda_parent_tag = [str(i.parent) for i in filing_soup.find_all(text=item_7_regex)]
                if len(mda_parent_tag)!=0:
                    mda_parent_tag_position = []
                    values_in_boundary = []
                    for i in mda_parent_tag:
                        mda_parent_tag_position.append(filing_soup_string.find(i))
                    for i in mda_parent_tag_position:
                        if i > mda_start_prev_tag[0] and i < mda_start_tag[0]:
                            values_in_boundary.append(i)
                        else:
                            continue
                    if len(values_in_boundary)==1:
                        filing_10k_mda_df.loc[idx, 'MDA_Start_Position'] = values_in_boundary[0]
                    else:
                        filing_10k_mda_df.loc[idx, 'MDA_Start_Position'] = 'N/A'
                else:
                    filing_10k_mda_df.loc[idx, 'MDA_Start_Position'] = 'N/A'
            else:
                filing_10k_mda_df.loc[idx, 'MDA_Start_Position'] = 'N/A'
        elif len(mda_start_tag)>1 and len(mda_start_prev_tag)==1:
            difference = []
            for count, i in enumerate(mda_start_prev_tag):
                difference.append(mda_start_prev_tag[0] - mda_start_tag[count])
            difference_neg = []
            for count, i in enumerate(difference):
                if i < 0:
                    difference_neg.append(i)
                else:
                    continue
            if len(difference_neg)>0:
                mda_start_tag = [mda_start_tag[difference.index(max(difference_neg))]]
                item_7_regex = re.compile(r'(?i)item\s*7(\.|\s*)')
                mda_parent_tag = [str(i.parent) for i in filing_soup.find_all(text=item_7_regex)]
                if len(mda_parent_tag)!=0:
                    mda_parent_tag_position = []
                    values_in_boundary = []
                    for i in mda_parent_tag:
                        mda_parent_tag_position.append(filing_soup_string.find(i))
                    for i in mda_parent_tag_position:
                        if i > mda_start_prev_tag[0] and i < mda_start_tag[0]:
                            values_in_boundary.append(i)
                        else:
                            continue
                    if len(values_in_boundary)==1:
                        filing_10k_mda_df.loc[idx, 'MDA_Start_Position'] = values_in_boundary[0]
                    else:
                        filing_10k_mda_df.loc[idx, 'MDA_Start_Position'] = 'N/A'
                else:
                    filing_10k_mda_df.loc[idx, 'MDA_Start_Position'] = 'N/A'
            else:
                filing_10k_mda_df.loc[idx, 'MDA_Start_Position'] = 'N/A'            
        else:
            filing_10k_mda_df.loc[idx, 'MDA_Start_Position'] = 'N/A'
        
        
        # occurrences of MDA_Page_Number_End_Tag in filing_soup_string
        mda_end_tag = [i.start() for i in re.finditer(row['MDA_Page_Number_End_Tag'], filing_soup_string)]
        # occurences of MDA_Page_Number_End_Prev_Tag in filing_soup_string
        mda_end_prev_tag = [i.start() for i in re.finditer(row['MDA_Page_Number_End_Prev_Tag'], filing_soup_string)]
        if len(mda_end_tag)==1 and len(mda_end_prev_tag)==1:
            item_7a_regex = re.compile(r'(?i)item\s*7a(\.|\s*)')
            mda_parent_tag = [str(i.parent) for i in filing_soup.find_all(text=item_7a_regex)]
            if len(mda_parent_tag)!=0:
                mda_parent_tag_position = []
                values_in_boundary = []
                for i in mda_parent_tag:
                    mda_parent_tag_position.append(filing_soup_string.find(i))
                for i in mda_parent_tag_position:
                    if i > mda_end_prev_tag[0] and i < mda_end_tag[0]:
                        values_in_boundary.append(i)
                    else:
                        continue
                if len(values_in_boundary)==1:
                    filing_10k_mda_df.loc[idx, 'MDA_End_Position'] = values_in_boundary[0]
                else:
                    filing_10k_mda_df.loc[idx, 'MDA_End_Position'] = 'N/A'
            else:
                filing_10k_mda_df.loc[idx, 'MDA_End_Position'] = 'N/A'
        elif len(mda_end_tag)==1 and len(mda_end_prev_tag)>1:
            difference = []
            for count, i in enumerate(mda_end_prev_tag):
                difference.append(mda_end_tag[0] - mda_end_prev_tag[count])
            difference_pos = []
            for count, i in enumerate(difference):
                if i > 0:
                    difference_pos.append(i)
                else:
                    continue
            if len(difference_pos)>0:
                mda_end_prev_tag = [mda_end_prev_tag[difference.index(min(difference_pos))]]  
                item_7a_regex = re.compile(r'(?i)item\s*7a(\.|\s*)')
                mda_parent_tag = [str(i.parent) for i in filing_soup.find_all(text=item_7a_regex)]
                if len(mda_parent_tag)!=0:
                    mda_parent_tag_position = []
                    values_in_boundary = []
                    for i in mda_parent_tag:
                        mda_parent_tag_position.append(filing_soup_string.find(i))
                    for i in mda_parent_tag_position:
                        if i > mda_end_prev_tag[0] and i < mda_end_tag[0]:
                            values_in_boundary.append(i)
                        else:
                            continue
                    if len(values_in_boundary)==1:
                        filing_10k_mda_df.loc[idx, 'MDA_End_Position'] = values_in_boundary[0]
                    else:
                        filing_10k_mda_df.loc[idx, 'MDA_End_Position'] = 'N/A'
                else:
                    filing_10k_mda_df.loc[idx, 'MDA_End_Position'] = 'N/A'
            else:
                filing_10k_mda_df.loc[idx, 'MDA_End_Position'] = 'N/A'
        elif len(mda_end_tag)>1 and len(mda_end_prev_tag)==1:
            difference = []
            for count, i in enumerate(mda_end_prev_tag):
                difference.append(mda_end_prev_tag[0] - mda_end_tag[count])
            difference_neg = []
            for count, i in enumerate(difference):
                if i < 0:
                    difference_neg.append(i)
                else:
                    continue
            if len(difference_neg)>0:
                mda_end_tag = [mda_end_tag[difference.index(max(difference_neg))]]
                item_7a_regex = re.compile(r'(?i)item\s*7a(\.|\s*)')
                mda_parent_tag = [str(i.parent) for i in filing_soup.find_all(text=item_7a_regex)]
                if len(mda_parent_tag)!=0:
                    mda_parent_tag_position = []
                    values_in_boundary = []
                    for i in mda_parent_tag:
                        mda_parent_tag_position.append(filing_soup_string.find(i))
                    for i in mda_parent_tag_position:
                        if i > mda_end_prev_tag[0] and i < mda_end_tag[0]:
                            values_in_boundary.append(i)
                        else:
                            continue
                    if len(values_in_boundary)==1:
                        filing_10k_mda_df.loc[idx, 'MDA_End_Position'] = values_in_boundary[0]
                    else:
                        filing_10k_mda_df.loc[idx, 'MDA_End_Position'] = 'N/A'
                else:
                    filing_10k_mda_df.loc[idx, 'MDA_End_Position'] = 'N/A'
            else:
                filing_10k_mda_df.loc[idx, 'MDA_End_Position'] = 'N/A'        
        else:
            filing_10k_mda_df.loc[idx, 'MDA_End_Position'] = 'N/A'
    else:
        filing_10k_mda_df.loc[idx, 'MDA_Start_Position'] = 'N/A'
        filing_10k_mda_df.loc[idx, 'MDA_End_Position'] = 'N/A'
    
    # provide user feedback 
    print('-------------------------------------------------------------------------------------')
    print(idx, row['Ticker_Symbol'])
    print('\n')
    print('md&a start position:',  filing_10k_mda_df.loc[idx, 'MDA_Start_Position'])
    print('md&a end position:', filing_10k_mda_df.loc[idx, 'MDA_End_Position'])
    
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 1 second
    time.sleep(1)  

-------------------------------------------------------------------------------------
0 AIR


md&a start position: 176725
md&a end position: 275637
-------------------------------------------------------------------------------------
1 AIR


md&a start position: 186736
md&a end position: 272962
-------------------------------------------------------------------------------------
2 AIR


md&a start position: 229060
md&a end position: 405617
-------------------------------------------------------------------------------------
3 AIR


md&a start position: 291382
md&a end position: 482986
-------------------------------------------------------------------------------------
4 AIR


md&a start position: 226707
md&a end position: 426552
-------------------------------------------------------------------------------------
5 HES


md&a start position: 408520
md&a end position: 729637
-------------------------------------------------------------------------------------
6 HES


md&a start positio

-------------------------------------------------------------------------------------
56 LUB


md&a start position: N/A
md&a end position: N/A
-------------------------------------------------------------------------------------
57 LUB


md&a start position: N/A
md&a end position: N/A
-------------------------------------------------------------------------------------
58 LUB


md&a start position: N/A
md&a end position: N/A
-------------------------------------------------------------------------------------
59 CATO


md&a start position: 431541
md&a end position: 565279
-------------------------------------------------------------------------------------
60 CATO


md&a start position: 412294
md&a end position: 551760
-------------------------------------------------------------------------------------
61 GCO


md&a start position: 412252
md&a end position: 730292
-------------------------------------------------------------------------------------
62 GCO


md&a start position: 411886

-------------------------------------------------------------------------------------
112 GLT


md&a start position: N/A
md&a end position: N/A
-------------------------------------------------------------------------------------
113 GLT


md&a start position: 595243
md&a end position: 1794312
-------------------------------------------------------------------------------------
114 GTN


md&a start position: 308186
md&a end position: 526617
-------------------------------------------------------------------------------------
115 GTN


md&a start position: 492054
md&a end position: 793544
-------------------------------------------------------------------------------------
116 GTN


md&a start position: 816230
md&a end position: 1085868
-------------------------------------------------------------------------------------
117 HHS


md&a start position: 415787
md&a end position: 646713
-------------------------------------------------------------------------------------
118 HP


md&a star

-------------------------------------------------------------------------------------
168 OSG


md&a start position: 696992
md&a end position: 980757
-------------------------------------------------------------------------------------
169 OMI


md&a start position: 288093
md&a end position: 607222
-------------------------------------------------------------------------------------
170 OMI


md&a start position: 317750
md&a end position: 648029
-------------------------------------------------------------------------------------
171 PKD


md&a start position: 385945
md&a end position: 631203
-------------------------------------------------------------------------------------
172 PKD


md&a start position: N/A
md&a end position: N/A
-------------------------------------------------------------------------------------
173 PKD


md&a start position: N/A
md&a end position: 896915
-------------------------------------------------------------------------------------
174 PKD


md&a start po

-------------------------------------------------------------------------------------
224 MRO


md&a start position: N/A
md&a end position: N/A
-------------------------------------------------------------------------------------
225 MRO


md&a start position: N/A
md&a end position: N/A
-------------------------------------------------------------------------------------
226 MRO


md&a start position: N/A
md&a end position: N/A
-------------------------------------------------------------------------------------
227 MRO


md&a start position: N/A
md&a end position: N/A
-------------------------------------------------------------------------------------
228 S


md&a start position: 408086
md&a end position: 1273415
-------------------------------------------------------------------------------------
229 S


md&a start position: 455680
md&a end position: 2051537
-------------------------------------------------------------------------------------
230 S


md&a start position: 442284
md&a

...

-------------------------------------------------------------------------------------
840 FET


md&a start position: 519843
md&a end position: 1144808
-------------------------------------------------------------------------------------
841 ORN


md&a start position: 483755
md&a end position: N/A
-------------------------------------------------------------------------------------
842 ORN


md&a start position: 414662
md&a end position: 628330
-------------------------------------------------------------------------------------
843 ORN


md&a start position: 511764
md&a end position: N/A
-------------------------------------------------------------------------------------
844 ORN


md&a start position: 423711
md&a end position: N/A
-------------------------------------------------------------------------------------
845 ORN


md&a start position: 425678
md&a end position: N/A
-------------------------------------------------------------------------------------
846 ORN


md&a start posi

-------------------------------------------------------------------------------------
896 SPLP


md&a start position: 315066
md&a end position: 898067
-------------------------------------------------------------------------------------
897 NE


md&a start position: 268563
md&a end position: 467433
-------------------------------------------------------------------------------------
898 NE


md&a start position: 413917
md&a end position: 726761
-------------------------------------------------------------------------------------
899 NE


md&a start position: 340929
md&a end position: 586075
-------------------------------------------------------------------------------------
900 NE


md&a start position: 302853
md&a end position: 554886
-------------------------------------------------------------------------------------
901 NE


md&a start position: 924058
md&a end position: 1773548
-------------------------------------------------------------------------------------
902 NE


md&a sta

-------------------------------------------------------------------------------------
952 TLYS


md&a start position: N/A
md&a end position: 724534
-------------------------------------------------------------------------------------
953 SLCA


md&a start position: N/A
md&a end position: N/A
-------------------------------------------------------------------------------------
954 ERA


md&a start position: 667438
md&a end position: N/A
-------------------------------------------------------------------------------------
955 ERA


md&a start position: N/A
md&a end position: N/A
-------------------------------------------------------------------------------------
956 ERA


md&a start position: 672052
md&a end position: N/A
-------------------------------------------------------------------------------------
957 ERA


md&a start position: 573227
md&a end position: N/A
-------------------------------------------------------------------------------------
958 LPI


md&a start position: N/A
m

-------------------------------------------------------------------------------------
1007 KN


md&a start position: 388626
md&a end position: 940038
-------------------------------------------------------------------------------------
1008 KN


md&a start position: 631573
md&a end position: 1174697
-------------------------------------------------------------------------------------
1009 CVEO


md&a start position: N/A
md&a end position: N/A
-------------------------------------------------------------------------------------
1010 CVEO


md&a start position: N/A
md&a end position: N/A
-------------------------------------------------------------------------------------
1011 CVEO


md&a start position: N/A
md&a end position: N/A
-------------------------------------------------------------------------------------
1012 ESI


md&a start position: 598257
md&a end position: 1366347
-------------------------------------------------------------------------------------
1013 ESI


md&a start p

-------------------------------------------------------------------------------------
1062 BNED


md&a start position: 486783
md&a end position: N/A
-------------------------------------------------------------------------------------
1063 AGR


md&a start position: 1777786
md&a end position: 3806857
-------------------------------------------------------------------------------------
1064 AGR


md&a start position: N/A
md&a end position: N/A
-------------------------------------------------------------------------------------
1065 GCI


md&a start position: 533461
md&a end position: N/A
-------------------------------------------------------------------------------------
1066 EXTN


md&a start position: 535257
md&a end position: 1041912
-------------------------------------------------------------------------------------
1067 HPE


md&a start position: N/A
md&a end position: N/A
-------------------------------------------------------------------------------------
1068 NBLX


md&a star

## 20-F Filings - Source Operating and Financial Review and Prospects Narrative Page Numbers (Start and End) 

In [8]:
# create a dataframe containing 20-F observations
# note: observations will be filtered from filing_page_df
filing_20f_ofp_df = filing_page_df.loc[filing_page_df['Filing_Type']=='20-F'].reset_index(drop=True)

# find page number on which operating and financial review and prospects narrative (management's discussion and analysis narrative) starts and ends
# note: the page numbers (start and end) will be stored in new columns of filing_20f_ofp_df
# note: the operating and financial review and prospects narrative is labeled as 'item 5' in 20-F filings
# note: the end of the operating and financial review and prospects narrative is the start of the next item (i.e. item 6)
filing_20f_ofp_df['OFP_Page_Number_Start'] = ''
filing_20f_ofp_df['OFP_Page_Number_End'] = ''
for idx, row in filing_20f_ofp_df.iterrows():
    filing_request = requests.get(url=row['Filing_Report_Url'])
    filing_soup = BeautifulSoup(filing_request.content, 'lxml')
    ofp_regex = re.compile(r'(?i)operating\s*(and|\&)\s*financial\s*review.*')
    dsme_regex = re.compile(r'(?i)directors(\,|.|)\s*senior\s*management\s*(and|\&)\s*emp.*')
    ofp_occurence = filing_soup.find_all(text=ofp_regex)
    dsme_occurence = filing_soup.find_all(text=dsme_regex)
    if len(ofp_occurence)!=0:
        ofp_table = None
        i = 0
        while ofp_table==None:
            if i==len(ofp_occurence):
                break
            else:
                ofp_contents_page_row = ofp_occurence[i].find_parent('tr')
                i += 1
                ofp_table = ofp_contents_page_row
        if ofp_contents_page_row!=None:
            ofp_contents_page_row_numbers = re.findall('\d+', ofp_contents_page_row.text)
            if len(ofp_contents_page_row_numbers)>1:
                for number in ofp_contents_page_row_numbers:
                    if number=='5':
                        continue
                    else:
                        filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start'] = number
                        break
            elif len(ofp_contents_page_row_numbers)==1 and ofp_contents_page_row_numbers[0]!='5':
                filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start'] = ofp_contents_page_row_numbers[0]
            else:
                filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start'] = 'N/A'
        else:
             filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start'] = 'N/A'
    else:
         filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start'] = 'N/A'
    
    if len(dsme_occurence)!=0:
        dsme_table = None
        i = 0
        while dsme_table==None:
            if i==len(dsme_occurence):
                break
            else:
                dsme_contents_page_row = dsme_occurence[i].find_parent('tr')
                i += 1
                dsme_table = dsme_contents_page_row
        if dsme_contents_page_row!=None:
            dsme_contents_page_row_numbers = re.findall('\d+', dsme_contents_page_row.text)
            if len(dsme_contents_page_row_numbers)>1:
                for number in dsme_contents_page_row_numbers:
                    if number=='6':
                        continue
                    else:
                        filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End'] = number
                        break
            elif len(dsme_contents_page_row_numbers)==1 and dsme_contents_page_row_numbers[0]!='6':
                filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End'] = dsme_contents_page_row_numbers[0]
            else:
                filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End'] = 'N/A'
        else:
            filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End'] = 'N/A'
    else:
        filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End'] = 'N/A'
        
    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(idx, row['Ticker_Symbol'])
    print('filing_date:', row['Filing_Date'])
    print('operating and financial review and prospects narrative starts on page:', filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start'])
    print('operating and financial review and prospects narrative ends on page:', filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End'])
    print('\n')
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 1 second 
    time.sleep(1)   

-------------------------------------------------------------------------------------
0 TK
filing_date: 2012-04-25
operating and financial review and prospects narrative starts on page: 34
operating and financial review and prospects narrative ends on page: 65


-------------------------------------------------------------------------------------
1 TK
filing_date: 2013-04-29
operating and financial review and prospects narrative starts on page: 34
operating and financial review and prospects narrative ends on page: 63


-------------------------------------------------------------------------------------
2 TK
filing_date: 2014-04-28
operating and financial review and prospects narrative starts on page: 34
operating and financial review and prospects narrative ends on page: 64


-------------------------------------------------------------------------------------
3 TK
filing_date: 2015-04-29
operating and financial review and prospects narrative starts on page: 34
operating and financia

-------------------------------------------------------------------------------------
31 CLS
filing_date: 2018-03-12
operating and financial review and prospects narrative starts on page: 44
operating and financial review and prospects narrative ends on page: 93


-------------------------------------------------------------------------------------
32 CLS
filing_date: 2019-03-11
operating and financial review and prospects narrative starts on page: 43
operating and financial review and prospects narrative ends on page: 91


-------------------------------------------------------------------------------------
33 TNP
filing_date: 2012-04-17
operating and financial review and prospects narrative starts on page: 45
operating and financial review and prospects narrative ends on page: 78


-------------------------------------------------------------------------------------
34 TNP
filing_date: 2013-04-29
operating and financial review and prospects narrative starts on page: 44
operating and 

-------------------------------------------------------------------------------------
62 SSW
filing_date: 2012-03-26
operating and financial review and prospects narrative starts on page: 50
operating and financial review and prospects narrative ends on page: 75


-------------------------------------------------------------------------------------
63 SSW
filing_date: 2013-03-19
operating and financial review and prospects narrative starts on page: 45
operating and financial review and prospects narrative ends on page: 73


-------------------------------------------------------------------------------------
64 SSW
filing_date: 2014-03-11
operating and financial review and prospects narrative starts on page: 46
operating and financial review and prospects narrative ends on page: 77


-------------------------------------------------------------------------------------
65 SSW
filing_date: 2015-03-10
operating and financial review and prospects narrative starts on page: 47
operating and 

-------------------------------------------------------------------------------------
93 CZZ
filing_date: 2013-07-31
operating and financial review and prospects narrative starts on page: 44
operating and financial review and prospects narrative ends on page: 78


-------------------------------------------------------------------------------------
94 CZZ
filing_date: 2014-04-30
operating and financial review and prospects narrative starts on page: 47
operating and financial review and prospects narrative ends on page: 75


-------------------------------------------------------------------------------------
95 CZZ
filing_date: 2015-04-30
operating and financial review and prospects narrative starts on page: 49
operating and financial review and prospects narrative ends on page: 84


-------------------------------------------------------------------------------------
96 CZZ
filing_date: 2016-04-29
operating and financial review and prospects narrative starts on page: 63
operating and 

-------------------------------------------------------------------------------------
124 TNK
filing_date: 2019-04-10
operating and financial review and prospects narrative starts on page: 41
operating and financial review and prospects narrative ends on page: 63


-------------------------------------------------------------------------------------
125 GSL
filing_date: 2012-04-13
operating and financial review and prospects narrative starts on page: 40
operating and financial review and prospects narrative ends on page: 54


-------------------------------------------------------------------------------------
126 GSL
filing_date: 2013-04-12
operating and financial review and prospects narrative starts on page: 41
operating and financial review and prospects narrative ends on page: 57


-------------------------------------------------------------------------------------
127 GSL
filing_date: 2014-04-22
operating and financial review and prospects narrative starts on page: 42
operating 

-------------------------------------------------------------------------------------
155 CMRE
filing_date: 2017-03-14
operating and financial review and prospects narrative starts on page: 56
operating and financial review and prospects narrative ends on page: 91


-------------------------------------------------------------------------------------
156 CMRE
filing_date: 2018-02-27
operating and financial review and prospects narrative starts on page: 56
operating and financial review and prospects narrative ends on page: 92


-------------------------------------------------------------------------------------
157 CMRE
filing_date: 2019-03-07
operating and financial review and prospects narrative starts on page: 59
operating and financial review and prospects narrative ends on page: 98


-------------------------------------------------------------------------------------
158 PACD
filing_date: 2014-03-04
operating and financial review and prospects narrative starts on page: 49
operat

-------------------------------------------------------------------------------------
186 ASC
filing_date: 2017-03-13
operating and financial review and prospects narrative starts on page: 49
operating and financial review and prospects narrative ends on page: 62


-------------------------------------------------------------------------------------
187 ASC
filing_date: 2018-03-29
operating and financial review and prospects narrative starts on page: 52
operating and financial review and prospects narrative ends on page: 65


-------------------------------------------------------------------------------------
188 ASC
filing_date: 2019-02-15
operating and financial review and prospects narrative starts on page: 53
operating and financial review and prospects narrative ends on page: 66


-------------------------------------------------------------------------------------
189 DLNG
filing_date: 2016-04-18
operating and financial review and prospects narrative starts on page: 74
operating

-------------------------------------------------------------------------------------
217 BBU
filing_date: 2019-03-18
operating and financial review and prospects narrative starts on page: 56
operating and financial review and prospects narrative ends on page: 92


-------------------------------------------------------------------------------------
218 SDRL
filing_date: 2015-04-21
operating and financial review and prospects narrative starts on page: 31
operating and financial review and prospects narrative ends on page: 57


-------------------------------------------------------------------------------------
219 SDRL
filing_date: 2016-04-28
operating and financial review and prospects narrative starts on page: 35
operating and financial review and prospects narrative ends on page: 56


-------------------------------------------------------------------------------------
220 SDRL
filing_date: 2017-04-27
operating and financial review and prospects narrative starts on page: 37
operati

## 20-F Filings - Manually Input Operating and Financial Review and Prospects Narrative Page Numbers (Start and End) for N/A Observations

In [9]:
# manually input N/A Observations for OFP_Page_Number_Start and OFP_Page_Number_End in filing_20f_ofp_df
filing_20f_ofp_df.loc[100, 'OFP_Page_Number_End'] = '86'

filing_20f_ofp_df.loc[101, 'OFP_Page_Number_End'] = '105'

filing_20f_ofp_df.loc[102, 'OFP_Page_Number_End'] = '98'

filing_20f_ofp_df.loc[103, 'OFP_Page_Number_Start'] = '69'
filing_20f_ofp_df.loc[103, 'OFP_Page_Number_End'] = '100'

filing_20f_ofp_df.loc[104, 'OFP_Page_Number_Start'] = '80'
filing_20f_ofp_df.loc[104, 'OFP_Page_Number_End'] = '114'

filing_20f_ofp_df.loc[105, 'OFP_Page_Number_End'] = '120'

filing_20f_ofp_df.loc[106, 'OFP_Page_Number_End'] = '144'

filing_20f_ofp_df.loc[107, 'OFP_Page_Number_End'] = '157'

filing_20f_ofp_df.loc[212, 'OFP_Page_Number_End'] = '99'

# the operation and financial review and prospects narrative for index 78 of filing_20f_ofp_df is contained in a separate exhibit (EX-99.3)
# remove this observation from filing_20f_ofp_df and amend correpsonding information in filing_page_df (observation is situated in index 993)
filing_20f_ofp_df = filing_20f_ofp_df.drop(index=78).reset_index(drop=True)
filing_page_df.loc[993, 'MDA_Placement'] = 'EX-99.3'
filing_page_df.loc[993, 'Filing_Report_Url'] = 'https://www.sec.gov/Archives/edgar/data/1368519/000136851916000024/noa20-fpart412x31x2015.htm'

## 20-F Filings - Store Previous Page Number for OFP_Page_Number_Start and OFP_Page_Number_End

In [10]:
# note: in order to identify a boundary of where the operating and financial review and prospects narrative begins and ends, 
#       it is necessary to store OFP_Page_Number_Start - 1 and OFP_Page_Number_End - 1 in filing_20f_ofp_df

# create a new column in filing_20f_ofp_df to store previous page number to OFP_Page_Number_Start 
filing_20f_ofp_df['OFP_Page_Number_Start_Prev'] = ''
# create a new column in filing_20f_ofp_df to store previous page number to OFP_Page_Number_End 
filing_20f_ofp_df['OFP_Page_Number_End_Prev'] = ''

# compute OFP_Page_Number_Start_Prev from OFP_Page_Number_Start
for idx, row in filing_20f_ofp_df.iterrows():
    if row['OFP_Page_Number_Start']!='N/A':
        filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start_Prev'] = str((int(float(row['OFP_Page_Number_Start'].strip())) - 1))
    else:
        filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start_Prev'] = 'N/A'
# compute OFP_Page_Number_End_Prev from OFP_Page_Number_End
for idx, row in filing_20f_ofp_df.iterrows():
    if row['OFP_Page_Number_End']!='N/A':
        filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End_Prev'] = str((int(float(row['OFP_Page_Number_End'].strip())) - 1))
    else:
        filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End_Prev'] = 'N/A'

## 20-F Filings - Check if Page Breaks Exist and Store OFP Start and End Tags

In [11]:
# note: in order to extract the operating and financial review and prospects narrative of each 20-F filing by page number,
#       a check on if the page breaks (separating pages) exist is necessary
# note: page breaks that separate pages in 20-F filings are usually denoted by a thematic break (<hr>)
# note: if a 20-F filing does not contain thematic breaks, the next best alternative is to find the tag style 'page-break-before:always'
# note: the alternative solution can be done via a regular expression

# create a new column in filing_20f_ofp_df to store a boolean value corresponding if page breaks exist or not
filing_20f_ofp_df['Page_Breaks'] = ''
# create a new column in filing_20f_ofp_df to store the tag in which the OFP narrative begins
filing_20f_ofp_df['OFP_Page_Number_Start_Tag'] = ''
# create a new column in filing_20f_ofp_df to store the tag of the previous page to where the OFP narrative begins
filing_20f_ofp_df['OFP_Page_Number_Start_Prev_Tag'] = ''
# create a new column in filing_20f_ofp_df to store the tag in which the OFP narrative ends
filing_20f_ofp_df['OFP_Page_Number_End_Tag'] = ''
# create a new column in filing_20f_ofp_df to store the tag of the previous page to where the OFP narrative ends
filing_20f_ofp_df['OFP_Page_Number_End_Prev_Tag'] = ''
# create a new column in filing_20f_ofp_df to store a boolean value corresponding to if OFP narrative start page matches retrieved value
filing_20f_ofp_df['OFP_Page_Number_Start_Match'] = ''
# create a new column in filing_20f_ofp_df to store a boolean value corresponding to if previous page to OFP start page matches retrieved value
filing_20f_ofp_df['OFP_Page_Number_Start_Prev_Match'] = ''
# create a new column in filing_20f_ofp_df to store a boolean value corresponding to if OFP narrative end page matches retrieved value
filing_20f_ofp_df['OFP_Page_Number_End_Match'] = ''
# create a new column in filing_20f_ofp_df to store a boolean value corresponding to if previous page to OFP end page matches retrieved value
filing_20f_ofp_df['OFP_Page_Number_End_Prev_Match'] = ''

# start page
for idx, row in filing_20f_ofp_df.iterrows():
    filing_request = requests.get(url=row['Filing_Report_Url'])
    filing_soup = BeautifulSoup(filing_request.content, 'lxml')
    if row['OFP_Page_Number_Start']!='N/A' and row['OFP_Page_Number_Start']!='N/A':
        page_break_tags_one = filing_soup.find_all('hr')     
        page_break_tags_two = filing_soup.find_all(style=re.compile(r'(?i).*page-break-(after|before)\s*:\s*always.*'))
        if len(page_break_tags_one)!=0:
            filing_20f_ofp_df.loc[idx, 'Page_Breaks'] = 'True'
            page_break_tag_counter = Counter(page_break_tags_one)
            most_common_tag = page_break_tag_counter.most_common(1)[0][0]
            for tag in page_break_tags_one:
                if tag==most_common_tag:
                    page_number = tag.find_previous(text=re.compile('^([^a-zA-Z0-9]|){1}\d{1,3}([^a-zA-Z0-9]|){1}$'))
                    if page_number!=None:
                        if page_number.strip()==row['OFP_Page_Number_Start'].strip():
                            filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start_Tag'] = str(page_number.parent)
                            break
                        else:
                            filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start_Tag'] = 'N/A'
                    else:
                        continue
                else:
                    pass
        elif len(page_break_tags_two)!=0:
            filing_20f_ofp_df.loc[idx, 'Page_Breaks'] = 'True'
            page_break_tag_counter = Counter(page_break_tags_two)
            most_common_tag = page_break_tag_counter.most_common(1)[0][0]
            for tag in page_break_tags_two:
                if tag==most_common_tag:
                    page_number = tag.find_previous(text=re.compile('^([^a-zA-Z0-9]|){1}\d{1,3}([^a-zA-Z0-9]|){1}$'))
                    if page_number!=None:
                        if page_number.strip()==row['OFP_Page_Number_Start'].strip():
                            filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start_Tag'] = str(page_number.parent)
                            break
                        else:
                            filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start_Tag'] = 'N/A'
                    else:
                        continue
                else:
                    pass
        else:
            filing_20f_ofp_df.loc[idx, 'Page_Breaks'] = 'False'
            filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start_Tag'] = 'N/A' 
    else:
        filing_20f_ofp_df.loc[idx, 'Page_Breaks'] = 'False'
        filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start_Tag'] = 'N/A'   
    
    if page_number.strip()==row['OFP_Page_Number_Start'].strip():
        filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start_Match'] = 'True'
    else:
        filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start_Match'] = 'False'

    # provide user feedback
    print('-------------------------------------------------------------------------------------')
    print(idx, row['Ticker_Symbol'])
    print('\n')
    print('start page:')
    print('filing_date:', row['Filing_Date'])
    print('page breaks exist:', filing_20f_ofp_df.loc[idx, 'Page_Breaks'])
    print('operating and financial review and prospects start page number tag:', page_number.parent.name)
    print('observed operating and financial review and prospects start page number:', row['OFP_Page_Number_Start'].strip())
    print('retrieved operating and financial review and prospects start page number:', page_number.strip())
    print('OFP page number match:', filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start_Match'])
    print('\n')
    
    
    # previous page of start page
    if row['OFP_Page_Number_Start']!='N/A' and row['OFP_Page_Number_Start']!='N/A':
        page_break_tags_one = filing_soup.find_all('hr')     
        page_break_tags_two = filing_soup.find_all(style=re.compile(r'(?i).*page-break-(after|before)\s*:\s*always.*'))
        if len(page_break_tags_one)!=0:
            page_break_tag_counter = Counter(page_break_tags_one)
            most_common_tag = page_break_tag_counter.most_common(1)[0][0]
            for tag in page_break_tags_one:
                if tag==most_common_tag:
                    page_number = tag.find_previous(text=re.compile('^([^a-zA-Z0-9]|){1}\d{1,3}([^a-zA-Z0-9]|){1}$'))
                    if page_number!=None:
                        if page_number.strip()==row['OFP_Page_Number_Start_Prev'].strip():
                            filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start_Prev_Tag'] = str(page_number.parent)
                            break
                        else:
                            filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start_Prev_Tag'] = 'N/A'
                    else:
                        continue
                else:
                    pass
        elif len(page_break_tags_two)!=0:
            page_break_tag_counter = Counter(page_break_tags_two)
            most_common_tag = page_break_tag_counter.most_common(1)[0][0]
            for tag in page_break_tags_two:
                if tag==most_common_tag:
                    page_number = tag.find_previous(text=re.compile('^([^a-zA-Z0-9]|){1}\d{1,3}([^a-zA-Z0-9]|){1}$'))
                    if page_number!=None:
                        if page_number.strip()==row['OFP_Page_Number_Start_Prev'].strip():
                            filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start_Prev_Tag'] = str(page_number.parent)
                            break
                        else:
                            filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start_Prev_Tag'] = 'N/A'
                    else:
                        continue
                else:
                    pass
        else:
            filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start_Prev_Tag'] = 'N/A' 
    else:
        filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start_Prev_Tag'] = 'N/A'   
    
    if page_number.strip()==row['OFP_Page_Number_Start_Prev'].strip():
        filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start_Prev_Match'] = 'True'
    else:
        filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start_Prev_Match'] = 'False'

    # provide user feedback
    print('previous page to start page:')
    print('operating and financial review and prospects previous page to start page number tag:', page_number.parent.name)
    print('observed operating and financial review and prospects previous page to  start page number:', row['OFP_Page_Number_Start_Prev'].strip())
    print('retrieved operating and financial review and prospects previous page to  start page number:', page_number.strip())
    print('OFP previous page to start page number match:', filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_Start_Prev_Match'])
    print('\n')
    
    
    # end page
    if row['OFP_Page_Number_Start']!='N/A' and row['OFP_Page_Number_End']!='N/A':
        page_break_tags_one = filing_soup.find_all('hr')     
        page_break_tags_two = filing_soup.find_all(style=re.compile(r'(?i).*page-break-(after|before)\s*:\s*always.*'))
        if len(page_break_tags_one)!=0:
            page_break_tag_counter = Counter(page_break_tags_one)
            most_common_tag = page_break_tag_counter.most_common(1)[0][0]
            for tag in page_break_tags_one:
                if tag==most_common_tag:
                    page_number = tag.find_previous(text=re.compile('^([^a-zA-Z0-9]|){1}\d{1,3}([^a-zA-Z0-9]|){1}$'))
                    if page_number!=None:
                        if page_number.strip()==row['OFP_Page_Number_End'].strip():
                            filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End_Tag'] = str(page_number.parent)
                            break
                        else:
                            filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End_Tag'] = 'N/A'
                    else:
                        continue
                else:
                    pass
        elif len(page_break_tags_two)!=0:
            page_break_tag_counter = Counter(page_break_tags_two)
            most_common_tag = page_break_tag_counter.most_common(1)[0][0]
            for tag in page_break_tags_two:
                if tag==most_common_tag:
                    page_number = tag.find_previous(text=re.compile('^([^a-zA-Z0-9]|){1}\d{1,3}([^a-zA-Z0-9]|){1}$'))
                    if page_number!=None:
                        if page_number.strip()==row['OFP_Page_Number_End'].strip():
                            filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End_Tag'] = str(page_number.parent)
                            break
                        else:
                            filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End_Tag'] = 'N/A'
                    else:
                        continue
                else:
                    pass
        else:
            filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End_Tag'] = 'N/A' 
    else:
        filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End_Tag'] = 'N/A'
    
    if page_number.strip()==row['OFP_Page_Number_End'].strip():
        filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End_Match'] = 'True'
    else:
        filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End_Match'] = 'False'
    
    # provide user feedback
    print('end page:')
    print('operating and financial review and prospects end page number tag:', page_number.parent.name)
    print('observed operating and financial review and prospects end page number:', row['OFP_Page_Number_End'].strip())
    print('retrieved operating and financial review and prospects end page number:', page_number.strip())
    print('OFP end page number match:', filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End_Match'])
    print('\n')
    
    
    # previous page to end page
    if row['OFP_Page_Number_Start']!='N/A' and row['OFP_Page_Number_End']!='N/A':
        page_break_tags_one = filing_soup.find_all('hr')     
        page_break_tags_two = filing_soup.find_all(style=re.compile(r'(?i).*page-break-(after|before)\s*:\s*always.*'))
        if len(page_break_tags_one)!=0:
            page_break_tag_counter = Counter(page_break_tags_one)
            most_common_tag = page_break_tag_counter.most_common(1)[0][0]
            for tag in page_break_tags_one:
                if tag==most_common_tag:
                    page_number = tag.find_previous(text=re.compile('^([^a-zA-Z0-9]|){1}\d{1,3}([^a-zA-Z0-9]|){1}$'))
                    if page_number!=None:
                        if page_number.strip()==row['OFP_Page_Number_End_Prev'].strip():
                            filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End_Prev_Tag'] = str(page_number.parent)
                            break
                        else:
                            filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End_Prev_Tag'] = 'N/A'
                    else:
                        continue
                else:
                    pass
        elif len(page_break_tags_two)!=0:
            page_break_tag_counter = Counter(page_break_tags_two)
            most_common_tag = page_break_tag_counter.most_common(1)[0][0]
            for tag in page_break_tags_two:
                if tag==most_common_tag:
                    page_number = tag.find_previous(text=re.compile('^([^a-zA-Z0-9]|){1}\d{1,3}([^a-zA-Z0-9]|){1}$'))
                    if page_number!=None:
                        if page_number.strip()==row['OFP_Page_Number_End_Prev'].strip():
                            filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End_Prev_Tag'] = str(page_number.parent)
                            break
                        else:
                            filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End_Prev_Tag'] = 'N/A'
                    else:
                        continue
                else:
                    pass
        else:
            filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End_Prev_Tag'] = 'N/A' 
    else:
        filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End_Prev_Tag'] = 'N/A'
    
    if page_number.strip()==row['OFP_Page_Number_End_Prev'].strip():
        filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End_Prev_Match'] = 'True'
    else:
        filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End_Prev_Match'] = 'False'
    
    # provide user feedback
    print('previous page to end page:')
    print('operating and financial review and prospects previous to end page number tag:', page_number.parent.name)
    print('observed operating and financial review and prospects previous to end page number:', row['OFP_Page_Number_End_Prev'].strip())
    print('retrieved operating and financial review and prospects previous to end page number:', page_number.strip())
    print('OFP previous to end page number match:', filing_20f_ofp_df.loc[idx, 'OFP_Page_Number_End_Prev_Match'])
    print('\n')
    
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 3 seconds 
    time.sleep(1)   

-------------------------------------------------------------------------------------
0 TK


start page:
filing_date: 2012-04-25
page breaks exist: True
operating and financial review and prospects start page number tag: font
observed operating and financial review and prospects start page number: 34
retrieved operating and financial review and prospects start page number: 34
OFP page number match: True


previous page to start page:
operating and financial review and prospects previous page to start page number tag: font
observed operating and financial review and prospects previous page to  start page number: 33
retrieved operating and financial review and prospects previous page to  start page number: 33
OFP previous page to start page number match: True


end page:
operating and financial review and prospects end page number tag: font
observed operating and financial review and prospects end page number: 65
retrieved operating and financial review and prospects end page number: 65


previous page to start page:
operating and financial review and prospects previous page to start page number tag: font
observed operating and financial review and prospects previous page to  start page number: 44
retrieved operating and financial review and prospects previous page to  start page number: 44
OFP previous page to start page number match: True


end page:
operating and financial review and prospects end page number tag: font
observed operating and financial review and prospects end page number: 87
retrieved operating and financial review and prospects end page number: 87
OFP end page number match: True


previous page to end page:
operating and financial review and prospects previous to end page number tag: font
observed operating and financial review and prospects previous to end page number: 86
retrieved operating and financial review and prospects previous to end page number: 86
OFP previous to end page number match: True


----------------------------------------------

end page:
operating and financial review and prospects end page number tag: font
observed operating and financial review and prospects end page number: 51
retrieved operating and financial review and prospects end page number: 51
OFP end page number match: True


previous page to end page:
operating and financial review and prospects previous to end page number tag: font
observed operating and financial review and prospects previous to end page number: 50
retrieved operating and financial review and prospects previous to end page number: 50
OFP previous to end page number match: True


-------------------------------------------------------------------------------------
13 CYD


start page:
filing_date: 2015-04-15
page breaks exist: True
operating and financial review and prospects start page number tag: font
observed operating and financial review and prospects start page number: 38
retrieved operating and financial review and prospects start page number: 38
OFP page number match: Tru

previous page to end page:
operating and financial review and prospects previous to end page number tag: font
observed operating and financial review and prospects previous to end page number: 46
retrieved operating and financial review and prospects previous to end page number: 46
OFP previous to end page number match: True


-------------------------------------------------------------------------------------
19 NAT


start page:
filing_date: 2013-03-19
page breaks exist: True
operating and financial review and prospects start page number tag: font
observed operating and financial review and prospects start page number: 33
retrieved operating and financial review and prospects start page number: 33
OFP page number match: True


previous page to start page:
operating and financial review and prospects previous page to start page number tag: font
observed operating and financial review and prospects previous page to  start page number: 32
retrieved operating and financial review and pr

-------------------------------------------------------------------------------------
25 CEA


start page:
filing_date: 2012-04-24
page breaks exist: True
operating and financial review and prospects start page number tag: td
observed operating and financial review and prospects start page number: 36
retrieved operating and financial review and prospects start page number: 36
OFP page number match: True


previous page to start page:
operating and financial review and prospects previous page to start page number tag: td
observed operating and financial review and prospects previous page to  start page number: 35
retrieved operating and financial review and prospects previous page to  start page number: 35
OFP previous page to start page number match: True


end page:
operating and financial review and prospects end page number tag: td
observed operating and financial review and prospects end page number: 51
retrieved operating and financial review and prospects end page number: 51
OFP 

...

previous page to end page:
operating and financial review and prospects previous to end page number tag: p
observed operating and financial review and prospects previous to end page number: 70
retrieved operating and financial review and prospects previous to end page number: 70
OFP previous to end page number match: True


-------------------------------------------------------------------------------------
193 NVGS


start page:
filing_date: 2017-03-01
page breaks exist: True
operating and financial review and prospects start page number tag: p
observed operating and financial review and prospects start page number: 52
retrieved operating and financial review and prospects start page number: 52
OFP page number match: True


previous page to start page:
operating and financial review and prospects previous page to start page number tag: p
observed operating and financial review and prospects previous page to  start page number: 51
retrieved operating and financial review and prospects

-------------------------------------------------------------------------------------
199 SALT


start page:
filing_date: 2019-03-20
page breaks exist: True
operating and financial review and prospects start page number tag: font
observed operating and financial review and prospects start page number: 48
retrieved operating and financial review and prospects start page number: 48
OFP page number match: True


previous page to start page:
operating and financial review and prospects previous page to start page number tag: font
observed operating and financial review and prospects previous page to  start page number: 47
retrieved operating and financial review and prospects previous page to  start page number: 47
OFP previous page to start page number match: True


end page:
operating and financial review and prospects end page number tag: font
observed operating and financial review and prospects end page number: 69
retrieved operating and financial review and prospects end page number:

-------------------------------------------------------------------------------------
205 GLOP


start page:
filing_date: 2019-02-26
page breaks exist: True
operating and financial review and prospects start page number tag: font
observed operating and financial review and prospects start page number: 69
retrieved operating and financial review and prospects start page number: 69
OFP page number match: True


previous page to start page:
operating and financial review and prospects previous page to start page number tag: font
observed operating and financial review and prospects previous page to  start page number: 68
retrieved operating and financial review and prospects previous page to  start page number: 68
OFP previous page to start page number match: True


end page:
operating and financial review and prospects end page number tag: font
observed operating and financial review and prospects end page number: 103
retrieved operating and financial review and prospects end page number

-------------------------------------------------------------------------------------
211 FCAU


start page:
filing_date: 2019-02-22
page breaks exist: True
operating and financial review and prospects start page number tag: font
observed operating and financial review and prospects start page number: 40
retrieved operating and financial review and prospects start page number: 40
OFP page number match: True


previous page to start page:
operating and financial review and prospects previous page to start page number tag: font
observed operating and financial review and prospects previous page to  start page number: 39
retrieved operating and financial review and prospects previous page to  start page number: 39
OFP previous page to start page number match: True


end page:
operating and financial review and prospects end page number tag: font
observed operating and financial review and prospects end page number: 99
retrieved operating and financial review and prospects end page number:

-------------------------------------------------------------------------------------
217 SDRL


start page:
filing_date: 2015-04-21
page breaks exist: True
operating and financial review and prospects start page number tag: font
observed operating and financial review and prospects start page number: 31
retrieved operating and financial review and prospects start page number: 31
OFP page number match: True


previous page to start page:
operating and financial review and prospects previous page to start page number tag: font
observed operating and financial review and prospects previous page to  start page number: 30
retrieved operating and financial review and prospects previous page to  start page number: 30
OFP previous page to start page number match: True


end page:
operating and financial review and prospects end page number tag: font
observed operating and financial review and prospects end page number: 57
retrieved operating and financial review and prospects end page number:

## 20-F Filings - Retrieve Position of OFP Start and End Tags in Fillings

In [12]:
# note: the operating and financial review and prospects narrative of each 20-F filing can be extracted by using the boundary of the start page,
#       previous to start page, end page, and previous to end page
# note: a regular expression can be used to find occurences of item 5 and item 6;
#       the position of the occurence within the given boundary will be stored
# note: if there is more than one occurence within the given boundary, these will be ignored due to time constraints

# create a new column in filing_20f_ofp_df to store the position of the start of the ofp narrative
filing_20f_ofp_df['OFP_Start_Position'] = ''
# create a new column in filing_20f_ofp_df to store the position of the end of the ofp narrative
filing_20f_ofp_df['OFP_End_Position'] = ''

# loop over observations in filing_20f_ofp_df
for idx, row in filing_20f_ofp_df.iterrows():
    if (row['OFP_Page_Number_Start_Match']!='False' and row['OFP_Page_Number_Start_Prev_Match']!='False' and
        row['OFP_Page_Number_End_Match']!='False' and row['OFP_Page_Number_End_Prev_Match']!='False'):
        filing_request = requests.get(url=row['Filing_Report_Url'])
        filing_soup = BeautifulSoup(filing_request.content, 'lxml')
        filing_soup_string = str(filing_soup)
        # start position
        # occurences of OFP_Page_Number_Start_Tag in filing_soup_string
        ofp_start_tag = [i.start() for i in re.finditer(row['OFP_Page_Number_Start_Tag'], filing_soup_string)]
        # occurences of OFP_Page_Number_Start_Prev_Tag in filing_soup_string
        ofp_start_prev_tag = [i.start() for i in re.finditer(row['OFP_Page_Number_Start_Prev_Tag'], filing_soup_string)]
        if len(ofp_start_tag)==1 and len(ofp_start_prev_tag)==1:
            item_5_regex = re.compile(r'(?i)item\s*5(\.|\s*)')
            ofp_parent_tag = [str(i.parent) for i in filing_soup.find_all(text=item_5_regex)]
            if len(ofp_parent_tag)!=0:
                ofp_parent_tag_position = []
                values_in_boundary = []
                for i in ofp_parent_tag:
                    ofp_parent_tag_position.append(filing_soup_string.find(i))
                for i in ofp_parent_tag_position:
                    if i > ofp_start_prev_tag[0] and i < ofp_start_tag[0]:
                        values_in_boundary.append(i)
                    else:
                        continue
                if len(values_in_boundary)==1:
                    filing_20f_ofp_df.loc[idx, 'OFP_Start_Position'] = values_in_boundary[0]
                else:
                    filing_20f_ofp_df.loc[idx, 'OFP_Start_Position'] = 'N/A'
            else:
                filing_20f_ofp_df.loc[idx, 'OFP_Start_Position'] = 'N/A'
        elif len(ofp_start_tag)==1 and len(ofp_start_prev_tag)>1:
            difference = []
            for count, i in enumerate(ofp_start_prev_tag):
                difference.append(ofp_start_tag[0] - ofp_start_prev_tag[count])
            difference_pos = []
            for count, i in enumerate(difference):
                if i > 0:
                    difference_pos.append(i)
                else:
                    continue
            if len(difference_pos)>0:
                ofp_start_prev_tag = [ofp_start_prev_tag[difference.index(min(difference_pos))]]                
                item_5_regex = re.compile(r'(?i)item\s*5(\.|\s*)')
                ofp_parent_tag = [str(i.parent) for i in filing_soup.find_all(text=item_5_regex)]
                if len(ofp_parent_tag)!=0:
                    ofp_parent_tag_position = []
                    values_in_boundary = []
                    for i in ofp_parent_tag:
                        ofp_parent_tag_position.append(filing_soup_string.find(i))
                    for i in ofp_parent_tag_position:
                        if i > ofp_start_prev_tag[0] and i < ofp_start_tag[0]:
                            values_in_boundary.append(i)
                        else:
                            continue
                    if len(values_in_boundary)==1:
                        filing_20f_ofp_df.loc[idx, 'OFP_Start_Position'] = values_in_boundary[0]
                    else:
                        filing_20f_ofp_df.loc[idx, 'OFP_Start_Position'] = 'N/A'
                else:
                    filing_20f_ofp_df.loc[idx, 'OFP_Start_Position'] = 'N/A'
            else:
                filing_20f_ofp_df.loc[idx, 'OFP_Start_Position'] = 'N/A'
        elif len(ofp_start_tag)>1 and len(ofp_start_prev_tag)==1:
            difference = []
            for count, i in enumerate(ofp_start_prev_tag):
                difference.append(ofp_start_prev_tag[0] - ofp_start_tag[count])
            difference_neg = []
            for count, i in enumerate(difference):
                if i < 0:
                    difference_neg.append(i)
                else:
                    continue
            if len(difference_neg)>0:
                ofp_start_tag = [ofp_start_tag[difference.index(max(difference_neg))]]
                item_5_regex = re.compile(r'(?i)item\s*5(\.|\s*)')
                ofp_parent_tag = [str(i.parent) for i in filing_soup.find_all(text=item_5_regex)]
                if len(ofp_parent_tag)!=0:
                    ofp_parent_tag_position = []
                    values_in_boundary = []
                    for i in ofp_parent_tag:
                        ofp_parent_tag_position.append(filing_soup_string.find(i))
                    for i in ofp_parent_tag_position:
                        if i > ofp_start_prev_tag[0] and i < ofp_start_tag[0]:
                            values_in_boundary.append(i)
                        else:
                            continue
                    if len(values_in_boundary)==1:
                        filing_20f_ofp_df.loc[idx, 'OFP_Start_Position'] = values_in_boundary[0]
                    else:
                        filing_20f_ofp_df.loc[idx, 'OFP_Start_Position'] = 'N/A'
                else:
                    filing_20f_ofp_df.loc[idx, 'OFP_Start_Position'] = 'N/A'
            else:
                filing_20f_ofp_df.loc[idx, 'OFP_Start_Position'] = 'N/A'            
        else:
            filing_20f_ofp_df.loc[idx, 'OFP_Start_Position'] = 'N/A'
        
        
        # occurrences of OFP_Page_Number_End_Tag in filing_soup_string
        ofp_end_tag = [i.start() for i in re.finditer(row['OFP_Page_Number_End_Tag'], filing_soup_string)]
        # occurences of OFP_Page_Number_End_Prev_Tag in filing_soup_string
        ofp_end_prev_tag = [i.start() for i in re.finditer(row['OFP_Page_Number_End_Prev_Tag'], filing_soup_string)]
        if len(ofp_end_tag)==1 and len(ofp_end_prev_tag)==1:
            item_6_regex = re.compile(r'(?i)item\s*6(\.|\s*)')
            ofp_parent_tag = [str(i.parent) for i in filing_soup.find_all(text=item_6_regex)]
            if len(ofp_parent_tag)!=0:
                ofp_parent_tag_position = []
                values_in_boundary = []
                for i in ofp_parent_tag:
                    ofp_parent_tag_position.append(filing_soup_string.find(i))
                for i in ofp_parent_tag_position:
                    if i > ofp_end_prev_tag[0] and i < ofp_end_tag[0]:
                        values_in_boundary.append(i)
                    else:
                        continue
                if len(values_in_boundary)==1:
                    filing_20f_ofp_df.loc[idx, 'OFP_End_Position'] = values_in_boundary[0]
                else:
                    filing_20f_ofp_df.loc[idx, 'OFP_End_Position'] = 'N/A'
            else:
                filing_20f_ofp_df.loc[idx, 'OFP_End_Position'] = 'N/A'
        elif len(ofp_end_tag)==1 and len(ofp_end_prev_tag)>1:
            difference = []
            for count, i in enumerate(ofp_end_prev_tag):
                difference.append(ofp_end_tag[0] - ofp_end_prev_tag[count])
            difference_pos = []
            for count, i in enumerate(difference):
                if i > 0:
                    difference_pos.append(i)
                else:
                    continue
            if len(difference_pos)>0:
                ofp_end_prev_tag = [ofp_end_prev_tag[difference.index(min(difference_pos))]]  
                item_6_regex = re.compile(r'(?i)item\s*6(\.|\s*)')
                ofp_parent_tag = [str(i.parent) for i in filing_soup.find_all(text=item_6_regex)]
                if len(ofp_parent_tag)!=0:
                    ofp_parent_tag_position = []
                    values_in_boundary = []
                    for i in ofp_parent_tag:
                        ofp_parent_tag_position.append(filing_soup_string.find(i))
                    for i in ofp_parent_tag_position:
                        if i > ofp_end_prev_tag[0] and i < ofp_end_tag[0]:
                            values_in_boundary.append(i)
                        else:
                            continue
                    if len(values_in_boundary)==1:
                        filing_20f_ofp_df.loc[idx, 'OFP_End_Position'] = values_in_boundary[0]
                    else:
                        filing_20f_ofp_df.loc[idx, 'OFP_End_Position'] = 'N/A'
                else:
                    filing_20f_ofp_df.loc[idx, 'OFP_End_Position'] = 'N/A'
            else:
                filing_20f_ofp_df.loc[idx, 'OFP_End_Position'] = 'N/A'
        elif len(ofp_end_tag)>1 and len(ofp_end_prev_tag)==1:
            difference = []
            for count, i in enumerate(ofp_end_prev_tag):
                difference.append(ofp_end_prev_tag[0] - ofp_end_tag[count])
            difference_neg = []
            for count, i in enumerate(difference):
                if i < 0:
                    difference_neg.append(i)
                else:
                    continue
            if len(difference_neg)>0:
                ofp_end_tag = [ofp_end_tag[difference.index(max(difference_neg))]]
                item_6_regex = re.compile(r'(?i)item\s*6(\.|\s*)')
                ofp_parent_tag = [str(i.parent) for i in filing_soup.find_all(text=item_6_regex)]
                if len(ofp_parent_tag)!=0:
                    ofp_parent_tag_position = []
                    values_in_boundary = []
                    for i in ofp_parent_tag:
                        ofp_parent_tag_position.append(filing_soup_string.find(i))
                    for i in ofp_parent_tag_position:
                        if i > ofp_end_prev_tag[0] and i < ofp_end_tag[0]:
                            values_in_boundary.append(i)
                        else:
                            continue
                    if len(values_in_boundary)==1:
                        filing_20f_ofp_df.loc[idx, 'OFP_End_Position'] = values_in_boundary[0]
                    else:
                        filing_20f_ofp_df.loc[idx, 'OFP_End_Position'] = 'N/A'
                else:
                    filing_20f_ofp_df.loc[idx, 'OFP_End_Position'] = 'N/A'
            else:
                filing_20f_ofp_df.loc[idx, 'OFP_End_Position'] = 'N/A'        
        else:
            filing_20f_ofp_df.loc[idx, 'OFP_End_Position'] = 'N/A'
    else:
        filing_20f_ofp_df.loc[idx, 'OFP_Start_Position'] = 'N/A'
        filing_20f_ofp_df.loc[idx, 'OFP_End_Position'] = 'N/A'
    
    # provide user feedback 
    print('-------------------------------------------------------------------------------------')
    print(idx, row['Ticker_Symbol'])
    print('\n')
    print('ofp start position:',  filing_20f_ofp_df.loc[idx, 'OFP_Start_Position'])
    print('ofp end position:', filing_20f_ofp_df.loc[idx, 'OFP_End_Position'])
    
    # note: the SEC strictly states that developers should not exceed 10 requests per second 
    # delay next request to EDGAR by 1 second
    time.sleep(1)  

-------------------------------------------------------------------------------------
0 TK


ofp start position: 565576
ofp end position: 1154045
-------------------------------------------------------------------------------------
1 TK


ofp start position: 568217
ofp end position: 1177746
-------------------------------------------------------------------------------------
2 TK


ofp start position: 483881
ofp end position: 979510
-------------------------------------------------------------------------------------
3 TK


ofp start position: 497565
ofp end position: 1096371
-------------------------------------------------------------------------------------
4 TK


ofp start position: 576580
ofp end position: 1288894
-------------------------------------------------------------------------------------
5 TK


ofp start position: 1050292
ofp end position: 2530256
-------------------------------------------------------------------------------------
6 TK


ofp start position: 998942
ofp 

-------------------------------------------------------------------------------------
56 DHT


ofp start position: 748147
ofp end position: 1015082
-------------------------------------------------------------------------------------
57 DHT


ofp start position: 588736
ofp end position: 901692
-------------------------------------------------------------------------------------
58 DHT


ofp start position: 430123
ofp end position: 620340
-------------------------------------------------------------------------------------
59 DHT


ofp start position: 420904
ofp end position: 623551
-------------------------------------------------------------------------------------
60 DHT


ofp start position: 482637
ofp end position: 685505
-------------------------------------------------------------------------------------
61 DHT


ofp start position: 447620
ofp end position: 649988
-------------------------------------------------------------------------------------
62 SSW


ofp start position: 65

-------------------------------------------------------------------------------------
112 NMM


ofp start position: 487225
ofp end position: 725271
-------------------------------------------------------------------------------------
113 NMM


ofp start position: 536117
ofp end position: 776277
-------------------------------------------------------------------------------------
114 NMM


ofp start position: 566897
ofp end position: 841081
-------------------------------------------------------------------------------------
115 NMM


ofp start position: 597269
ofp end position: 954146
-------------------------------------------------------------------------------------
116 NMM


ofp start position: 603305
ofp end position: 958260
-------------------------------------------------------------------------------------
117 TNK


ofp start position: 549847
ofp end position: 767251
-------------------------------------------------------------------------------------
118 TNK


ofp start positi

-------------------------------------------------------------------------------------
168 BEP


ofp start position: N/A
ofp end position: 3322649
-------------------------------------------------------------------------------------
169 BEP


ofp start position: N/A
ofp end position: 4142917
-------------------------------------------------------------------------------------
170 BEP


ofp start position: N/A
ofp end position: 3846814
-------------------------------------------------------------------------------------
171 BEP


ofp start position: N/A
ofp end position: N/A
-------------------------------------------------------------------------------------
172 BEP


ofp start position: N/A
ofp end position: N/A
-------------------------------------------------------------------------------------
173 GLOG


ofp start position: 623590
ofp end position: 930505
-------------------------------------------------------------------------------------
174 GLOG


ofp start position: 638917
ofp e

## 40-F Filings - Source Management's Discussion and Analysis Narrative Page Numbers (Start and End)

In [13]:
# note: observations with MDA_Placement in 40-F filings in filing_page_df have no obvious pattern to identify their corresponding
#       management's discussion and analysis narrative
# note: these filings will not be retrieved for the reason outlined above
# note: observations of this kind total 5 rows in filing_page_df

# create a dataframe containing 40-F observations (where MDA_Placement = 40-F)
# note: observations will be filtered from filing_page_df
filing_40f_mda_df = filing_page_df.loc[filing_page_df['MDA_Placement']=='40-F'].reset_index(drop=True)

# create columns for the management's discussion and analysis start and end page numbers
# as mentioned above, these will be set to 'N/A'
filing_40f_mda_df['MDA_Page_Number_Start'] = 'N/A'
filing_40f_mda_df['MDA_Page_Number_End'] = 'N/A'

## Save filing_10k_mda_df, filing_20f_ofp_df, and filing_40f_mda_df to CSV Files

In [14]:
# save filing_10k_mda_df to csv file
filing_10k_mda_df.to_csv('filing_10k_mda_df.csv', index=False)

# save filing_20f_ofp_df to csv file
filing_20f_ofp_df.to_csv('filing_20f_ofp_df.csv', index=False)

# save filing_40f_mda_df to csv file
filing_40f_mda_df.to_csv('filing_40f_ofp_df.csv', index=False)